# Journey North Adult Sightings

In [1]:
#  Import and read
import pandas as pd 
import numpy as np
import json
# Sue Mottet 2-19-2022

# geoJSONifier

In [2]:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

# Load file

In [3]:
JNorth1996_2020_df = pd.read_csv("UWMadisonArb_JNorth_Project_MonarchsAndMilkweed_1996-2020.csv")
JNorth1996_2020_df.head()

C:\Users\motte\anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,sighting_date,species,number,latitude,longitude,flag_location,comments,school,grade,image_url,customerid,number_old,flag_number,species_old,flag_species
0,850841703,1996-12-16,Monarch Adult Sighted,1,27.594,-81.362,0,On the ballfield a solitary monarch butterfly ...,Walker memorial junior academy,10.0,-999999.999,135,-999999.999,1,Monarch Butterfly,1
1,851824801,1996-12-06,Monarch Adult Sighted,1,30.420,-87.224,0,Our 5th grade Discovery students who went alon...,Hambrick elementary school,5.0,-999999.999,150,-999999.999,1,Monarch Butterfly,1
2,854409931,1996-09-21,Monarch Adult Sighted,1,40.817,-96.710,0,We saw several hundred monarchs in our school ...,-999999.999,5.0,-999999.999,9701054,-999999.999,1,Monarch Butterfly,1
3,854557206,1996-09-21,Monarch Adult Sighted,1,40.817,-96.710,0,Over a hundered monarchs in our school butterf...,Fredstrom elementary school,6.0,-999999.999,175,-999999.999,1,Monarch Butterfly,1
4,855061862,1997-01-01,Monarch Adult Sighted,1,34.244,-77.946,0,"I Thought I Saw It, But I Was Suprised To See ...",Gregory,3.0,-999999.999,181,-999999.999,1,Monarch Butterfly,1


# Data Exploratory

In [4]:
JNorth1996_2020_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227433 entries, 0 to 227432
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             227433 non-null  int64  
 1   sighting_date  227433 non-null  object 
 2   species        227433 non-null  object 
 3   number         227433 non-null  int64  
 4   latitude       227433 non-null  float64
 5   longitude      227433 non-null  float64
 6   flag_location  227433 non-null  int64  
 7   comments       227433 non-null  object 
 8   school         227433 non-null  object 
 9   grade          227433 non-null  float64
 10  image_url      227433 non-null  object 
 11  customerid     227433 non-null  int64  
 12  number_old     227433 non-null  float64
 13  flag_number    227433 non-null  int64  
 14  species_old    227433 non-null  object 
 15  flag_species   227433 non-null  int64  
dtypes: float64(4), int64(6), object(6)
memory usage: 27.8+ MB


In [5]:
str(JNorth1996_2020_df.columns.tolist())

"['id', 'sighting_date', 'species', 'number', 'latitude', 'longitude', 'flag_location', 'comments', 'school', 'grade', 'image_url', 'customerid', 'number_old', 'flag_number', 'species_old', 'flag_species']"

In [6]:
JNorth1996_2020_df.describe()

,id,number,latitude,longitude,flag_location,grade,customerid,number_old,flag_number,flag_species
count,2.274330e+05,2.274330e+05,227433.000000,227433.000000,227433.000000,227433.000000,2.274330e+05,2.274330e+05,227433.000000,227433.000000
mean,1.420650e+09,1.186529e+02,38.400690,-87.569965,0.598018,-907386.217597,9.451737e+06,-2.844270e+05,0.294816,0.130324
std,1.456887e+08,2.209080e+04,5.268651,10.880362,0.490299,289894.342744,2.193961e+06,4.518136e+05,0.467089,0.336660
min,8.508417e+08,-1.700000e+01,-12.100000,-165.107000,0.000000,-999999.999000,0.000000e+00,-1.000000e+06,0.000000,0.000000
25%,1.377751e+09,1.000000e+00,34.723000,-95.268000,0.000000,-999999.999000,9.941050e+06,-1.000000e+06,0.000000,0.000000
50%,1.477448e+09,1.000000e+00,39.503000,-86.787000,1.000000,-999999.999000,9.977663e+06,1.000000e+00,0.000000,0.000000
75%,1.526383e+09,3.000000e+00,42.409000,-79.255000,1.000000,-999999.999000,1.000029e+07,3.000000e+00,1.000000,0.000000
max,1.526525e+09,1.000000e+07,65.136000,-52.883000,1.000000,1112.000000,1.004256e+07,1.000000e+07,2.000000,1.000000


In [7]:
JNorth1996_2020_df.columns.tolist()

['id',
 'sighting_date',
 'species',
 'number',
 'latitude',
 'longitude',
 'flag_location',
 'comments',
 'school',
 'grade',
 'image_url',
 'customerid',
 'number_old',
 'flag_number',
 'species_old',
 'flag_species']

In [8]:
JNorth1996_2020_df.shape

(227433, 16)

In [9]:
# Determine the number of unique values in each column.
for col in JNorth1996_2020_df:
  print(JNorth1996_2020_df[col].unique())

[ 850841703  851824801  854409931 ... 1526524404 1526524405 1526525021]
['1996-12-16' '1996-12-06' '1996-09-21' ... '2020-12-29' '2020-12-30'
 '2020-12-31']
['Monarch Adult Sighted' 'Monarch Adult (FIRST sighted)'
 'Monarch Egg (FIRST sighted)' 'Monarch (OTHER Observations)'
 'Monarch Larva (FIRST sighted)' 'Monarch PEAK Migration'
 'Milkweed (FIRST sighted)' 'Monarch Fall Roost' 'Monarch Egg Sighted'
 'Monarch Larva Sighted' 'Monarch, Captive-Reared' 'Milkweed Sighted']
[       1      150       20       89       30     2000       50     7500
       23      200        5      300        4      100       71      120
      325       21       28     5000    20000       36       25        6
       54     1000      160       24       60       37        2     3000
  1000000       10  2000000       40       85       65       15        8
      500       14      112       75       80    15000       18    10000
      912   100000      400      800       12       35      600       16
    25000    

In [10]:
# Determine the number of unique values in each column.
for col in JNorth1996_2020_df:
  print(JNorth1996_2020_df[col].value_counts())

850841703     1
1526350827    1
1526350816    1
1526350817    1
1526350818    1
             ..
1432512422    1
1432519704    1
1432562236    1
1432562641    1
1526525021    1
Name: id, Length: 227433, dtype: int64
2018-08-22    392
2018-10-03    377
2018-09-06    353
2019-09-19    337
2018-09-12    335
             ... 
2015-12-31      1
2015-11-24      1
2015-11-22      1
2006-07-30      1
1996-12-16      1
Name: sighting_date, Length: 7491, dtype: int64
Monarch Adult Sighted            102306
Monarch Adult (FIRST sighted)     36409
Monarch Larva Sighted             17378
Monarch, Captive-Reared           14339
Monarch (OTHER Observations)      13581
Monarch Egg Sighted                8736
Milkweed (FIRST sighted)           8465
Monarch PEAK Migration             7689
Monarch Egg (FIRST sighted)        6876
Monarch Fall Roost                 5035
Monarch Larva (FIRST sighted)      4898
Milkweed Sighted                   1721
Name: species, dtype: int64
1       139319
2        24169
3

In [11]:
# Find null values
for column in JNorth1996_2020_df.columns:
   print(f"Column {column} has {JNorth1996_2020_df[column].isnull().sum()} null values")

Column id has 0 null values
Column sighting_date has 0 null values
Column species has 0 null values
Column number has 0 null values
Column latitude has 0 null values
Column longitude has 0 null values
Column flag_location has 0 null values
Column comments has 0 null values
Column school has 0 null values
Column grade has 0 null values
Column image_url has 0 null values
Column customerid has 0 null values
Column number_old has 0 null values
Column flag_number has 0 null values
Column species_old has 0 null values
Column flag_species has 0 null values


In [12]:
# Find duplicate entries
print(f"Duplicate entries: {JNorth1996_2020_df.duplicated().sum()}")

Duplicate entries: 0


In [13]:
# filtering by type of observation (Monarch Adult Sighted)
JNorth_Adult_Site_df = JNorth1996_2020_df[JNorth1996_2020_df['species']=='Monarch Adult Sighted']
JNorth_Adult_Site_df.head()

,id,sighting_date,species,number,latitude,longitude,flag_location,comments,school,grade,image_url,customerid,number_old,flag_number,species_old,flag_species
0,850841703,1996-12-16,Monarch Adult Sighted,1,27.594,-81.362,0,On the ballfield a solitary monarch butterfly ...,Walker memorial junior academy,10.0,-999999.999,135,-999999.999,1,Monarch Butterfly,1
1,851824801,1996-12-06,Monarch Adult Sighted,1,30.420,-87.224,0,Our 5th grade Discovery students who went alon...,Hambrick elementary school,5.0,-999999.999,150,-999999.999,1,Monarch Butterfly,1
2,854409931,1996-09-21,Monarch Adult Sighted,1,40.817,-96.710,0,We saw several hundred monarchs in our school ...,-999999.999,5.0,-999999.999,9701054,-999999.999,1,Monarch Butterfly,1
3,854557206,1996-09-21,Monarch Adult Sighted,1,40.817,-96.710,0,Over a hundered monarchs in our school butterf...,Fredstrom elementary school,6.0,-999999.999,175,-999999.999,1,Monarch Butterfly,1
4,855061862,1997-01-01,Monarch Adult Sighted,1,34.244,-77.946,0,"I Thought I Saw It, But I Was Suprised To See ...",Gregory,3.0,-999999.999,181,-999999.999,1,Monarch Butterfly,1


In [14]:
JNorth_Adult_Site_df.to_csv('JNorth_Adult_Site.csv', header=True, mode='w' )

In [15]:
JNorth_Adult_Site_df.shape

(102306, 16)

# Format file for geoJSON 

In [17]:
JNorth_Adult_Site_df = pd.DataFrame(JNorth_Adult_Site_df)
JNorth_Adult_Site_df.drop(['id','flag_location', 'comments','school','grade','image_url','customerid','number_old','flag_number','species_old','flag_species'], axis=1, inplace=True)
JNorth_Adult_Site_df.head()

KeyError: "['id' 'flag_location' 'comments' 'school' 'grade' 'image_url' 'customerid'\n 'number_old' 'flag_number' 'species_old' 'flag_species'] not found in axis"

In [18]:
cols = ['sighting_date', 'species', 'number', 'latitude', 'longitude']

In [19]:
JNorth_Adult_Site_df =JNorth_Adult_Site_df.groupby(['sighting_date', 'species', 'latitude', 'longitude']).sum().reset_index()
JNorth_Adult_Site_df.head()

,sighting_date,species,latitude,longitude,number
0,1996-03-26,Monarch Adult Sighted,35.170,-79.396,1
1,1996-06-16,Monarch Adult Sighted,44.957,-93.540,1
2,1996-09-21,Monarch Adult Sighted,40.817,-96.710,2
3,1996-12-06,Monarch Adult Sighted,30.420,-87.224,1
4,1996-12-16,Monarch Adult Sighted,27.594,-81.362,1


In [20]:
JNorth_Adult_Site_df.columns.tolist()

['sighting_date', 'species', 'latitude', 'longitude', 'number']

In [21]:
JNorth_Adult_Site_df.head()

,sighting_date,species,latitude,longitude,number
0,1996-03-26,Monarch Adult Sighted,35.170,-79.396,1
1,1996-06-16,Monarch Adult Sighted,44.957,-93.540,1
2,1996-09-21,Monarch Adult Sighted,40.817,-96.710,2
3,1996-12-06,Monarch Adult Sighted,30.420,-87.224,1
4,1996-12-16,Monarch Adult Sighted,27.594,-81.362,1


In [22]:
JNorth_Adult_Site_df.shape

(98258, 5)

In [23]:
JNorth_Adult_Site_year_df = JNorth_Adult_Site_df
JNorth_Adult_Site_year_df['year'] = pd.DatetimeIndex(JNorth_Adult_Site_df['sighting_date']).year

# 2011

In [24]:
JNorth_Adult_Site_2011_df = JNorth_Adult_Site_year_df[JNorth_Adult_Site_year_df['year']==2011]
JNorth_Adult_Site_2011_df.tail()

,sighting_date,species,latitude,longitude,number,year
8960,2011-12-29,Monarch Adult Sighted,29.990,-90.255,2,2011
8961,2011-12-29,Monarch Adult Sighted,30.340,-81.831,10,2011
8962,2011-12-30,Monarch Adult Sighted,26.988,-82.343,25,2011
8963,2011-12-30,Monarch Adult Sighted,29.957,-90.078,1520,2011
8964,2011-12-30,Monarch Adult Sighted,34.346,-119.329,2,2011


In [25]:
JNorth_Adult_Site_2011_df.shape

(4087, 6)

In [26]:
JNorth_Adult_Site_2011_df = pd.DataFrame(JNorth_Adult_Site_2011_df)
JNorth_Adult_Site_2011_df.drop(['year'], axis=1, inplace=True)

C:\Users\motte\anaconda3\envs\mlenv\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [28]:
JNorth_Adult_Site_2011_df['month'] = pd.DatetimeIndex(JNorth_Adult_Site_2011_df['sighting_date']).month

C:\Users\motte\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## January

In [29]:
JNorth_Adult_Site_2011_month_df =pd.DataFrame(JNorth_Adult_Site_2011_df)
JNorth_Adult_Site_2011_1_df = JNorth_Adult_Site_2011_month_df[JNorth_Adult_Site_2011_month_df['month']==1]
JNorth_Adult_Site_2011_1_df.tail()

,sighting_date,species,latitude,longitude,number,month
4920,2011-01-30,Monarch Adult Sighted,27.770,-82.637,2,1
4921,2011-01-30,Monarch Adult Sighted,27.839,-82.715,1,1
4922,2011-01-30,Monarch Adult Sighted,29.582,-95.038,1,1
4923,2011-01-30,Monarch Adult Sighted,36.579,-121.832,1,1
4924,2011-01-31,Monarch Adult Sighted,30.862,-87.712,1,1


In [30]:
JNorth_Adult_Site_2011_1_df.shape

(47, 6)

In [31]:
JNorth_Adult_Site_2011_1_df = pd.DataFrame(JNorth_Adult_Site_2011_1_df)
JNorth_Adult_Site_2011_1_df.drop(['month'], axis=1, inplace=True)

In [32]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2011_1_df['latitude'] = JNorth_Adult_Site_2011_1_df['latitude'].astype(float)
JNorth_Adult_Site_2011_1_df['longitude'] = JNorth_Adult_Site_2011_1_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [33]:
geojson = df_to_geojson(JNorth_Adult_Site_2011_1_df, cols)

In [34]:
file = open("JNorth_Adult_Site_2011_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## February

In [35]:
JNorth_Adult_Site_2011_month_df =pd.DataFrame(JNorth_Adult_Site_2011_df)
JNorth_Adult_Site_2011_2_df = JNorth_Adult_Site_2011_month_df[JNorth_Adult_Site_2011_month_df['month']==2]
JNorth_Adult_Site_2011_2_df.tail()

,sighting_date,species,latitude,longitude,number,month
4956,2011-02-25,Monarch Adult Sighted,28.140,-97.006,1,2
4957,2011-02-25,Monarch Adult Sighted,36.579,-121.832,1,2
4958,2011-02-27,Monarch Adult Sighted,29.469,-98.528,1,2
4959,2011-02-28,Monarch Adult Sighted,26.530,-81.826,1,2
4960,2011-02-28,Monarch Adult Sighted,29.582,-95.038,1,2


In [36]:
JNorth_Adult_Site_2011_2_df.shape

(36, 6)

In [37]:
JNorth_Adult_Site_2011_2_df= pd.DataFrame(JNorth_Adult_Site_2011_2_df)
JNorth_Adult_Site_2011_2_df.drop(['month'], axis=1, inplace=True)

In [38]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2011_2_df['latitude'] = JNorth_Adult_Site_2011_2_df['latitude'].astype(float)
JNorth_Adult_Site_2011_2_df['longitude'] = JNorth_Adult_Site_2011_2_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [39]:
geojson = df_to_geojson(JNorth_Adult_Site_2011_2_df, cols)

In [40]:
file = open("JNorth_Adult_Site_2011_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## March

In [41]:
JNorth_Adult_Site_2011_month_df =pd.DataFrame(JNorth_Adult_Site_2011_df)
JNorth_Adult_Site_2011_3_df = JNorth_Adult_Site_2011_month_df[JNorth_Adult_Site_2011_month_df['month']==3]
JNorth_Adult_Site_2011_3_df.tail()

,sighting_date,species,latitude,longitude,number,month
4965,2011-03-05,Monarch Adult Sighted,36.579,-121.832,1,3
4966,2011-03-10,Monarch Adult Sighted,33.451,-112.069,1,3
4967,2011-03-12,Monarch Adult Sighted,35.221,-120.635,1,3
4968,2011-03-12,Monarch Adult Sighted,36.579,-121.832,1,3
4969,2011-03-18,Monarch Adult Sighted,35.221,-120.635,1,3


In [42]:
JNorth_Adult_Site_2011_3_df.shape

(9, 6)

In [43]:
JNorth_Adult_Site_2011_3_df =pd.DataFrame(JNorth_Adult_Site_2011_3_df)
JNorth_Adult_Site_2011_3_df.drop(['month'], axis=1, inplace=True)

In [44]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2011_3_df['latitude'] = JNorth_Adult_Site_2011_3_df['latitude'].astype(float)
JNorth_Adult_Site_2011_3_df['longitude'] = JNorth_Adult_Site_2011_3_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [45]:
geojson = df_to_geojson(JNorth_Adult_Site_2011_3_df, cols)

In [46]:
file = open("JNorth_Adult_Site_2011_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## April

In [47]:
JNorth_Adult_Site_2011_month_df =pd.DataFrame(JNorth_Adult_Site_2011_df)
JNorth_Adult_Site_2011_4_df = JNorth_Adult_Site_2011_month_df[JNorth_Adult_Site_2011_month_df['month']==4]
JNorth_Adult_Site_2011_4_df.tail()

,sighting_date,species,latitude,longitude,number,month
4970,2011-04-02,Monarch Adult Sighted,31.918,-81.024,1,4
4971,2011-04-07,Monarch Adult Sighted,19.557,-155.966,1,4
4972,2011-04-15,Monarch Adult Sighted,32.641,-107.068,3,4


In [48]:
JNorth_Adult_Site_2011_4_df.shape

(3, 6)

In [49]:
JNorth_Adult_Site_2011_4_df = pd.DataFrame(JNorth_Adult_Site_2011_4_df)
JNorth_Adult_Site_2011_4_df.drop(['month'], axis=1, inplace=True)

In [50]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2011_4_df['latitude'] = JNorth_Adult_Site_2011_4_df['latitude'].astype(float)
JNorth_Adult_Site_2011_4_df['longitude'] = JNorth_Adult_Site_2011_4_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [51]:
geojson = df_to_geojson(JNorth_Adult_Site_2011_4_df, cols)

In [52]:
file = open("JNorth_Adult_Site_2011_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## May

In [53]:
JNorth_Adult_Site_2011_month_df =pd.DataFrame(JNorth_Adult_Site_2011_df)
JNorth_Adult_Site_2011_5_df = JNorth_Adult_Site_2011_month_df[JNorth_Adult_Site_2011_month_df['month']==5]
JNorth_Adult_Site_2011_5_df.tail()

,sighting_date,species,latitude,longitude,number,month
4973,2011-05-15,Monarch Adult Sighted,33.846,-118.01,1,5


In [54]:
JNorth_Adult_Site_2011_5_df.shape

(1, 6)

In [55]:
JNorth_Adult_Site_2011_5_df =pd.DataFrame(JNorth_Adult_Site_2011_5_df)
JNorth_Adult_Site_2011_5_df.drop(['month'], axis=1, inplace=True)

In [56]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2011_5_df['latitude'] = JNorth_Adult_Site_2011_5_df['latitude'].astype(float)
JNorth_Adult_Site_2011_5_df['longitude'] = JNorth_Adult_Site_2011_5_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [57]:
geojson = df_to_geojson(JNorth_Adult_Site_2011_5_df, cols)

In [58]:
file = open("JNorth_Adult_Site_2011_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## June

In [59]:
JNorth_Adult_Site_2011_month_df =pd.DataFrame(JNorth_Adult_Site_2011_df)
JNorth_Adult_Site_2011_6_df = JNorth_Adult_Site_2011_month_df[JNorth_Adult_Site_2011_month_df['month']==6]
JNorth_Adult_Site_2011_6_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [60]:
JNorth_Adult_Site_2011_6_df.shape

(0, 6)

In [61]:
JNorth_Adult_Site_2011_6_df=pd.DataFrame(JNorth_Adult_Site_2011_6_df)
JNorth_Adult_Site_2011_6_df.drop(['month'], axis=1, inplace=True)

In [62]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2011_6_df['latitude'] = JNorth_Adult_Site_2011_6_df['latitude'].astype(float)
JNorth_Adult_Site_2011_6_df['longitude'] = JNorth_Adult_Site_2011_6_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [63]:
geojson = df_to_geojson(JNorth_Adult_Site_2011_6_df, cols)

In [64]:
file = open("JNorth_Adult_Site_2011_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## July

In [65]:
JNorth_Adult_Site_2011_month_df =pd.DataFrame(JNorth_Adult_Site_2011_df)
JNorth_Adult_Site_2011_7_df = JNorth_Adult_Site_2011_month_df[JNorth_Adult_Site_2011_month_df['month']==7]
JNorth_Adult_Site_2011_7_df.tail()

,sighting_date,species,latitude,longitude,number,month
4976,2011-07-20,Monarch Adult Sighted,40.880,-73.004,1,7
4977,2011-07-22,Monarch Adult Sighted,36.736,-105.476,1,7
4978,2011-07-23,Monarch Adult Sighted,44.850,-65.300,1,7
4979,2011-07-29,Monarch Adult Sighted,44.856,-92.249,1,7
4980,2011-07-31,Monarch Adult Sighted,44.420,-69.724,1,7


In [66]:
JNorth_Adult_Site_2011_7_df.shape

(7, 6)

In [67]:
JNorth_Adult_Site_2011_7_df = pd.DataFrame(JNorth_Adult_Site_2011_7_df)
JNorth_Adult_Site_2011_7_df.drop(['month'], axis=1, inplace=True)

In [68]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2011_7_df['latitude'] = JNorth_Adult_Site_2011_7_df['latitude'].astype(float)
JNorth_Adult_Site_2011_7_df['longitude'] = JNorth_Adult_Site_2011_7_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [69]:
geojson = df_to_geojson(JNorth_Adult_Site_2011_7_df, cols)

In [70]:
file = open("JNorth_Adult_Site_2011_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## August

In [71]:
JNorth_Adult_Site_2011_month_df =pd.DataFrame(JNorth_Adult_Site_2011_df)
JNorth_Adult_Site_2011_8_df = JNorth_Adult_Site_2011_month_df[JNorth_Adult_Site_2011_month_df['month']==8]
JNorth_Adult_Site_2011_8_df.tail()

,sighting_date,species,latitude,longitude,number,month
5565,2011-08-31,Monarch Adult Sighted,44.547,-103.721,1,8
5566,2011-08-31,Monarch Adult Sighted,44.902,-93.288,30,8
5567,2011-08-31,Monarch Adult Sighted,46.924,-96.714,10,8
5568,2011-08-31,Monarch Adult Sighted,47.112,-91.857,5,8
5569,2011-08-31,Monarch Adult Sighted,48.061,-101.422,1,8


In [72]:
JNorth_Adult_Site_2011_8_df.shape

(589, 6)

In [73]:
JNorth_Adult_Site_2011_8_df = pd.DataFrame(JNorth_Adult_Site_2011_8_df)
JNorth_Adult_Site_2011_8_df.drop(['month'], axis=1, inplace=True)

In [74]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2011_8_df['latitude'] = JNorth_Adult_Site_2011_8_df['latitude'].astype(float)
JNorth_Adult_Site_2011_8_df['longitude'] = JNorth_Adult_Site_2011_8_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [75]:
geojson = df_to_geojson(JNorth_Adult_Site_2011_8_df, cols)

In [76]:
file = open("JNorth_Adult_Site_2011_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## September

In [77]:
JNorth_Adult_Site_2011_month_df =pd.DataFrame(JNorth_Adult_Site_2011_df)
JNorth_Adult_Site_2011_9_df = JNorth_Adult_Site_2011_month_df[JNorth_Adult_Site_2011_month_df['month']==9]
JNorth_Adult_Site_2011_9_df.tail()

,sighting_date,species,latitude,longitude,number,month
7199,2011-09-30,Monarch Adult Sighted,44.300,-77.950,4,9
7200,2011-09-30,Monarch Adult Sighted,44.591,-69.343,1,9
7201,2011-09-30,Monarch Adult Sighted,44.800,-79.717,1,9
7202,2011-09-30,Monarch Adult Sighted,44.823,-70.340,1,9
7203,2011-09-30,Monarch Adult Sighted,44.845,-70.379,1,9


In [78]:
JNorth_Adult_Site_2011_9_df.shape

(1634, 6)

In [79]:
JNorth_Adult_Site_2011_9_df = pd.DataFrame(JNorth_Adult_Site_2011_9_df)
JNorth_Adult_Site_2011_9_df.drop(['month'], axis=1, inplace=True)

In [80]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2011_9_df['latitude'] = JNorth_Adult_Site_2011_9_df['latitude'].astype(float)
JNorth_Adult_Site_2011_9_df['longitude'] = JNorth_Adult_Site_2011_9_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [81]:
geojson = df_to_geojson(JNorth_Adult_Site_2011_9_df, cols)

In [82]:
file = open("JNorth_Adult_Site_2011_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## October

In [83]:
JNorth_Adult_Site_2011_month_df =pd.DataFrame(JNorth_Adult_Site_2011_df)
JNorth_Adult_Site_2011_10_df = JNorth_Adult_Site_2011_month_df[JNorth_Adult_Site_2011_month_df['month']==10]
JNorth_Adult_Site_2011_10_df.tail()

,sighting_date,species,latitude,longitude,number,month
8635,2011-10-31,Monarch Adult Sighted,36.070,-79.766,1,10
8636,2011-10-31,Monarch Adult Sighted,39.114,-74.821,1,10
8637,2011-10-31,Monarch Adult Sighted,39.255,-74.608,6,10
8638,2011-10-31,Monarch Adult Sighted,43.316,-73.935,1,10
8639,2011-10-31,Monarch Adult Sighted,43.900,-79.133,1,10


In [84]:
JNorth_Adult_Site_2011_10_df.shape

(1436, 6)

In [85]:
JNorth_Adult_Site_2011_10_df = pd.DataFrame(JNorth_Adult_Site_2011_10_df)
JNorth_Adult_Site_2011_10_df.drop(['month'], axis=1, inplace=True)

In [86]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2011_10_df['latitude'] = JNorth_Adult_Site_2011_10_df['latitude'].astype(float)
JNorth_Adult_Site_2011_10_df['longitude'] = JNorth_Adult_Site_2011_10_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [87]:
geojson = df_to_geojson(JNorth_Adult_Site_2011_10_df, cols)

In [88]:
file = open("JNorth_Adult_Site_2011_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## November

In [89]:
JNorth_Adult_Site_2011_month_df =pd.DataFrame(JNorth_Adult_Site_2011_df)
JNorth_Adult_Site_2011_11_df = JNorth_Adult_Site_2011_month_df[JNorth_Adult_Site_2011_month_df['month']==11]
JNorth_Adult_Site_2011_11_df.tail()

,sighting_date,species,latitude,longitude,number,month
8918,2011-11-29,Monarch Adult Sighted,29.887,-97.830,1,11
8919,2011-11-29,Monarch Adult Sighted,37.072,-76.478,1,11
8920,2011-11-30,Monarch Adult Sighted,28.233,-97.019,50,11
8921,2011-11-30,Monarch Adult Sighted,29.582,-95.038,5,11
8922,2011-11-30,Monarch Adult Sighted,29.635,-95.635,1,11


In [90]:
JNorth_Adult_Site_2011_11_df.shape

(283, 6)

In [91]:
JNorth_Adult_Site_2011_11_df = pd.DataFrame(JNorth_Adult_Site_2011_11_df)
JNorth_Adult_Site_2011_11_df.drop(['month'], axis=1, inplace=True)

In [92]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2011_11_df['latitude'] = JNorth_Adult_Site_2011_11_df['latitude'].astype(float)
JNorth_Adult_Site_2011_11_df['longitude'] = JNorth_Adult_Site_2011_11_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [93]:
geojson = df_to_geojson(JNorth_Adult_Site_2011_11_df, cols)

In [94]:
file = open("JNorth_Adult_Site_2011_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## December

In [95]:
JNorth_Adult_Site_2011_month_df =pd.DataFrame(JNorth_Adult_Site_2011_df)
JNorth_Adult_Site_2011_12_df = JNorth_Adult_Site_2011_month_df[JNorth_Adult_Site_2011_month_df['month']==12]
JNorth_Adult_Site_2011_12_df.tail()

,sighting_date,species,latitude,longitude,number,month
8960,2011-12-29,Monarch Adult Sighted,29.990,-90.255,2,12
8961,2011-12-29,Monarch Adult Sighted,30.340,-81.831,10,12
8962,2011-12-30,Monarch Adult Sighted,26.988,-82.343,25,12
8963,2011-12-30,Monarch Adult Sighted,29.957,-90.078,1520,12
8964,2011-12-30,Monarch Adult Sighted,34.346,-119.329,2,12


In [96]:
JNorth_Adult_Site_2011_12_df.shape

(42, 6)

In [97]:
JNorth_Adult_Site_2011_12_df = pd.DataFrame(JNorth_Adult_Site_2011_12_df)
JNorth_Adult_Site_2011_12_df.drop(['month'], axis=1, inplace=True)

In [98]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2011_12_df['latitude'] = JNorth_Adult_Site_2011_12_df['latitude'].astype(float)
JNorth_Adult_Site_2011_12_df['longitude'] = JNorth_Adult_Site_2011_12_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [99]:
geojson = df_to_geojson(JNorth_Adult_Site_2011_12_df, cols)

In [100]:
file = open("JNorth_Adult_Site_2011_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# 2012

In [101]:
JNorth_Adult_Site_2012_df = JNorth_Adult_Site_year_df[JNorth_Adult_Site_year_df['year']==2012]
JNorth_Adult_Site_2012_df.tail()

,sighting_date,species,latitude,longitude,number,year
12344,2012-12-26,Monarch Adult Sighted,34.272,-119.231,1,2012
12345,2012-12-26,Monarch Adult Sighted,34.423,-119.890,1,2012
12346,2012-12-26,Monarch Adult Sighted,35.132,-120.635,1,2012
12347,2012-12-26,Monarch Adult Sighted,36.626,-121.930,1,2012
12348,2012-12-27,Monarch Adult Sighted,37.063,-122.162,7500,2012


In [102]:
JNorth_Adult_Site_2012_df.shape

(3384, 6)

In [103]:
JNorth_Adult_Site_2012_df = pd.DataFrame(JNorth_Adult_Site_2012_df)
JNorth_Adult_Site_2012_df.drop(['year'], axis=1, inplace=True)

In [104]:
JNorth_Adult_Site_2012_df.tail

<bound method NDFrame.tail of       sighting_date                species  latitude  longitude  number
8965     2012-01-01  Monarch Adult Sighted    19.590   -100.264       1
8966     2012-01-01  Monarch Adult Sighted    19.670   -100.280       1
8967     2012-01-01  Monarch Adult Sighted    27.522    -82.578       1
8968     2012-01-01  Monarch Adult Sighted    29.556    -95.726       1
8969     2012-01-01  Monarch Adult Sighted    29.757    -95.365       1
...             ...                    ...       ...        ...     ...
12344    2012-12-26  Monarch Adult Sighted    34.272   -119.231       1
12345    2012-12-26  Monarch Adult Sighted    34.423   -119.890       1
12346    2012-12-26  Monarch Adult Sighted    35.132   -120.635       1
12347    2012-12-26  Monarch Adult Sighted    36.626   -121.930       1
12348    2012-12-27  Monarch Adult Sighted    37.063   -122.162    7500

[3384 rows x 5 columns]>

In [105]:
JNorth_Adult_Site_2012_df['month'] = pd.DatetimeIndex(JNorth_Adult_Site_2012_df['sighting_date']).month

## January

In [106]:
JNorth_Adult_Site_2012_month_df =pd.DataFrame(JNorth_Adult_Site_2012_df)
JNorth_Adult_Site_2012_1_df = JNorth_Adult_Site_2012_month_df[JNorth_Adult_Site_2012_month_df['month']==1]
JNorth_Adult_Site_2012_1_df.tail()

,sighting_date,species,latitude,longitude,number,month
9076,2012-01-30,Monarch Adult Sighted,33.324,-111.867,1,1
9077,2012-01-31,Monarch Adult Sighted,29.809,-91.221,1,1
9078,2012-01-31,Monarch Adult Sighted,29.990,-90.255,1,1
9079,2012-01-31,Monarch Adult Sighted,32.216,-110.971,1,1
9080,2012-01-31,Monarch Adult Sighted,33.804,-112.888,1,1


In [107]:
JNorth_Adult_Site_2012_1_df.shape

(116, 6)

In [108]:
JNorth_Adult_Site_2012_1_df = pd.DataFrame(JNorth_Adult_Site_2012_1_df)
JNorth_Adult_Site_2012_1_df.drop(['month'], axis=1, inplace=True)

In [109]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2012_1_df['latitude'] = JNorth_Adult_Site_2012_1_df['latitude'].astype(float)
JNorth_Adult_Site_2012_1_df['longitude'] = JNorth_Adult_Site_2012_1_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [110]:
geojson = df_to_geojson(JNorth_Adult_Site_2012_1_df, cols)

In [111]:
file = open("JNorth_Adult_Site_2012_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## February

In [112]:
JNorth_Adult_Site_2012_month_df =pd.DataFrame(JNorth_Adult_Site_2012_df)
JNorth_Adult_Site_2012_2_df = JNorth_Adult_Site_2012_month_df[JNorth_Adult_Site_2012_month_df['month']==2]
JNorth_Adult_Site_2012_2_df.tail()

,sighting_date,species,latitude,longitude,number,month
9114,2012-02-25,Monarch Adult Sighted,10.433,-85.400,1,2
9115,2012-02-25,Monarch Adult Sighted,29.676,-98.570,1,2
9116,2012-02-26,Monarch Adult Sighted,27.792,-80.479,1,2
9117,2012-02-26,Monarch Adult Sighted,29.676,-98.570,1,2
9118,2012-02-27,Monarch Adult Sighted,29.888,-95.563,1,2


In [113]:
JNorth_Adult_Site_2012_2_df.shape

(38, 6)

In [114]:
JNorth_Adult_Site_2012_2_df = pd.DataFrame(JNorth_Adult_Site_2012_2_df)
JNorth_Adult_Site_2012_2_df.drop(['month'], axis=1, inplace=True)

In [115]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2012_2_df['latitude'] = JNorth_Adult_Site_2012_2_df['latitude'].astype(float)
JNorth_Adult_Site_2012_2_df['longitude'] = JNorth_Adult_Site_2012_2_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [116]:
geojson = df_to_geojson(JNorth_Adult_Site_2012_2_df, cols)

In [117]:
file = open("JNorth_Adult_Site_2012_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## March

In [118]:
JNorth_Adult_Site_2012_month_df =pd.DataFrame(JNorth_Adult_Site_2012_df)
JNorth_Adult_Site_2012_3_df = JNorth_Adult_Site_2012_month_df[JNorth_Adult_Site_2012_month_df['month']==3]
JNorth_Adult_Site_2012_3_df.tail()

,sighting_date,species,latitude,longitude,number,month
9125,2012-03-15,Monarch Adult Sighted,26.623,-81.883,1,3
9126,2012-03-16,Monarch Adult Sighted,29.957,-90.078,1,3
9127,2012-03-18,Monarch Adult Sighted,30.212,-84.357,2,3
9128,2012-03-19,Monarch Adult Sighted,28.036,-81.928,1,3
9129,2012-03-21,Monarch Adult Sighted,27.839,-82.715,1,3


In [119]:
JNorth_Adult_Site_2012_3_df.shape

(11, 6)

In [120]:
JNorth_Adult_Site_2012_3_df = pd.DataFrame(JNorth_Adult_Site_2012_3_df)
JNorth_Adult_Site_2012_3_df.drop(['month'], axis=1, inplace=True)

In [121]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2012_3_df['latitude'] = JNorth_Adult_Site_2012_3_df['latitude'].astype(float)
JNorth_Adult_Site_2012_3_df['longitude'] = JNorth_Adult_Site_2012_3_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [122]:
geojson = df_to_geojson(JNorth_Adult_Site_2012_3_df, cols)

In [123]:
file = open("JNorth_Adult_Site_2012_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## April

In [124]:
JNorth_Adult_Site_2012_month_df =pd.DataFrame(JNorth_Adult_Site_2012_df)
JNorth_Adult_Site_2012_4_df = JNorth_Adult_Site_2012_month_df[JNorth_Adult_Site_2012_month_df['month']==4]
JNorth_Adult_Site_2012_4_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [125]:
JNorth_Adult_Site_2012_4_df.shape

(0, 6)

In [126]:
JNorth_Adult_Site_2012_4_df = pd.DataFrame(JNorth_Adult_Site_2012_4_df)
JNorth_Adult_Site_2012_4_df.drop(['month'], axis=1, inplace=True)

In [127]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2012_4_df['latitude'] = JNorth_Adult_Site_2012_4_df['latitude'].astype(float)
JNorth_Adult_Site_2012_4_df['longitude'] = JNorth_Adult_Site_2012_4_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [128]:
geojson = df_to_geojson(JNorth_Adult_Site_2012_4_df, cols)

In [129]:
file = open("JNorth_Adult_Site_2012_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## May

In [130]:
JNorth_Adult_Site_2012_month_df =pd.DataFrame(JNorth_Adult_Site_2012_df)
JNorth_Adult_Site_2012_5_df = JNorth_Adult_Site_2012_month_df[JNorth_Adult_Site_2012_month_df['month']==5]
JNorth_Adult_Site_2012_5_df.tail()

,sighting_date,species,latitude,longitude,number,month
9130,2012-05-08,Monarch Adult Sighted,41.886,-87.623,1,5
9131,2012-05-18,Monarch Adult Sighted,30.212,-84.357,5,5


In [131]:
JNorth_Adult_Site_2012_5_df.shape

(2, 6)

In [132]:
JNorth_Adult_Site_2012_5_df = pd.DataFrame(JNorth_Adult_Site_2012_5_df)
JNorth_Adult_Site_2012_5_df.drop(['month'], axis=1, inplace=True)

In [133]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2012_5_df['latitude'] = JNorth_Adult_Site_2012_5_df['latitude'].astype(float)
JNorth_Adult_Site_2012_5_df['longitude'] = JNorth_Adult_Site_2012_5_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [134]:
geojson = df_to_geojson(JNorth_Adult_Site_2012_5_df, cols)

In [135]:
file = open("JNorth_Adult_Site_2012_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## June

In [136]:
JNorth_Adult_Site_2012_month_df =pd.DataFrame(JNorth_Adult_Site_2012_df)
JNorth_Adult_Site_2012_6_df = JNorth_Adult_Site_2012_month_df[JNorth_Adult_Site_2012_month_df['month']==6]
JNorth_Adult_Site_2012_6_df.tail()

,sighting_date,species,latitude,longitude,number,month
9132,2012-06-18,Monarch Adult Sighted,49.733,-104.017,3,6
9133,2012-06-30,Monarch Adult Sighted,44.729,-91.869,1,6
9134,2012-06-30,Monarch Adult Sighted,46.183,-59.950,1,6


In [137]:
JNorth_Adult_Site_2012_6_df.shape

(3, 6)

In [138]:
JNorth_Adult_Site_2012_6_df = pd.DataFrame(JNorth_Adult_Site_2012_6_df)
JNorth_Adult_Site_2012_6_df.drop(['month'], axis=1, inplace=True)

In [139]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2012_6_df['latitude'] = JNorth_Adult_Site_2012_6_df['latitude'].astype(float)
JNorth_Adult_Site_2012_6_df['longitude'] = JNorth_Adult_Site_2012_6_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [140]:
geojson = df_to_geojson(JNorth_Adult_Site_2012_6_df, cols)

In [141]:
file = open("JNorth_Adult_Site_2012_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## July

In [142]:
JNorth_Adult_Site_2012_month_df =pd.DataFrame(JNorth_Adult_Site_2012_df)
JNorth_Adult_Site_2012_7_df = JNorth_Adult_Site_2012_month_df[JNorth_Adult_Site_2012_month_df['month']==7]
JNorth_Adult_Site_2012_7_df.tail()

,sighting_date,species,latitude,longitude,number,month
9138,2012-07-23,Monarch Adult Sighted,34.390,-111.430,1,7
9139,2012-07-25,Monarch Adult Sighted,42.830,-77.724,2,7
9140,2012-07-26,Monarch Adult Sighted,33.324,-111.867,1,7
9141,2012-07-26,Monarch Adult Sighted,37.119,-76.546,1,7
9142,2012-07-28,Monarch Adult Sighted,34.155,-109.446,1,7


In [143]:
JNorth_Adult_Site_2012_7_df.shape

(8, 6)

In [144]:
JNorth_Adult_Site_2012_7_df = pd.DataFrame(JNorth_Adult_Site_2012_7_df)
JNorth_Adult_Site_2012_7_df.drop(['month'], axis=1, inplace=True)

In [145]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2012_7_df['latitude'] = JNorth_Adult_Site_2012_7_df['latitude'].astype(float)
JNorth_Adult_Site_2012_7_df['longitude'] = JNorth_Adult_Site_2012_7_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [146]:
geojson = df_to_geojson(JNorth_Adult_Site_2012_7_df, cols)

In [147]:
file = open("JNorth_Adult_Site_2012_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## August

In [148]:
JNorth_Adult_Site_2012_month_df =pd.DataFrame(JNorth_Adult_Site_2012_df)
JNorth_Adult_Site_2012_8_df = JNorth_Adult_Site_2012_month_df[JNorth_Adult_Site_2012_month_df['month']==8]
JNorth_Adult_Site_2012_8_df.tail()

,sighting_date,species,latitude,longitude,number,month
9941,2012-08-31,Monarch Adult Sighted,44.000,-79.467,6,8
9942,2012-08-31,Monarch Adult Sighted,44.017,-79.817,5,8
9943,2012-08-31,Monarch Adult Sighted,44.417,-76.600,2,8
9944,2012-08-31,Monarch Adult Sighted,44.883,-93.283,2,8
9945,2012-08-31,Monarch Adult Sighted,45.100,-64.417,1,8


In [149]:
JNorth_Adult_Site_2012_8_df.shape

(803, 6)

In [150]:
JNorth_Adult_Site_2012_8_df = pd.DataFrame(JNorth_Adult_Site_2012_8_df)
JNorth_Adult_Site_2012_8_df.drop(['month'], axis=1, inplace=True)

In [151]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2012_8_df['latitude'] = JNorth_Adult_Site_2012_8_df['latitude'].astype(float)
JNorth_Adult_Site_2012_8_df['longitude'] = JNorth_Adult_Site_2012_8_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [152]:
geojson = df_to_geojson(JNorth_Adult_Site_2012_8_df, cols)

In [153]:
file = open("JNorth_Adult_Site_2012_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## September

In [154]:
JNorth_Adult_Site_2012_month_df =pd.DataFrame(JNorth_Adult_Site_2012_df)
JNorth_Adult_Site_2012_9_df = JNorth_Adult_Site_2012_month_df[JNorth_Adult_Site_2012_month_df['month']==9]
JNorth_Adult_Site_2012_9_df.tail()

,sighting_date,species,latitude,longitude,number,month
11301,2012-09-30,Monarch Adult Sighted,45.161,-93.235,2,9
11302,2012-09-30,Monarch Adult Sighted,45.561,-94.162,1,9
11303,2012-09-30,Monarch Adult Sighted,46.001,-94.341,2,9
11304,2012-09-30,Monarch Adult Sighted,47.523,-94.745,1,9
11305,2012-09-30,Monarch Adult Sighted,52.550,-113.640,1,9


In [155]:
JNorth_Adult_Site_2012_9_df.shape

(1360, 6)

In [156]:
JNorth_Adult_Site_2012_9_df = pd.DataFrame(JNorth_Adult_Site_2012_9_df)
JNorth_Adult_Site_2012_9_df.drop(['month'], axis=1, inplace=True)

In [157]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2012_9_df['latitude'] = JNorth_Adult_Site_2012_9_df['latitude'].astype(float)
JNorth_Adult_Site_2012_9_df['longitude'] = JNorth_Adult_Site_2012_9_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [158]:
geojson = df_to_geojson(JNorth_Adult_Site_2012_9_df, cols)

In [159]:
file = open("JNorth_Adult_Site_2012_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## October

In [160]:
JNorth_Adult_Site_2012_month_df =pd.DataFrame(JNorth_Adult_Site_2012_df)
JNorth_Adult_Site_2012_10_df = JNorth_Adult_Site_2012_month_df[JNorth_Adult_Site_2012_month_df['month']==10]
JNorth_Adult_Site_2012_10_df.tail()

,sighting_date,species,latitude,longitude,number,month
12184,2012-10-31,Monarch Adult Sighted,33.279,-82.114,3,10
12185,2012-10-31,Monarch Adult Sighted,33.285,-91.020,1,10
12186,2012-10-31,Monarch Adult Sighted,34.138,-118.166,4,10
12187,2012-10-31,Monarch Adult Sighted,34.391,-101.712,1,10
12188,2012-10-31,Monarch Adult Sighted,44.648,-83.361,1,10


In [161]:
JNorth_Adult_Site_2012_10_df.shape

(883, 6)

In [162]:
JNorth_Adult_Site_2012_10_df = pd.DataFrame(JNorth_Adult_Site_2012_10_df)
JNorth_Adult_Site_2012_10_df.drop(['month'], axis=1, inplace=True)

In [163]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2012_10_df['latitude'] = JNorth_Adult_Site_2012_10_df['latitude'].astype(float)
JNorth_Adult_Site_2012_10_df['longitude'] = JNorth_Adult_Site_2012_10_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [164]:
geojson = df_to_geojson(JNorth_Adult_Site_2012_10_df, cols)

In [165]:
file = open("JNorth_Adult_Site_2012_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## November

In [166]:
JNorth_Adult_Site_2012_month_df =pd.DataFrame(JNorth_Adult_Site_2012_df)
JNorth_Adult_Site_2012_11_df = JNorth_Adult_Site_2012_month_df[JNorth_Adult_Site_2012_month_df['month']==11]
JNorth_Adult_Site_2012_11_df.tail()

,sighting_date,species,latitude,longitude,number,month
12316,2012-11-26,Monarch Adult Sighted,30.340,-81.831,3,11
12317,2012-11-27,Monarch Adult Sighted,30.358,-81.421,1,11
12318,2012-11-28,Monarch Adult Sighted,29.836,-95.735,1,11
12319,2012-11-28,Monarch Adult Sighted,32.216,-110.971,2,11
12320,2012-11-30,Monarch Adult Sighted,30.230,-97.714,1,11


In [167]:
JNorth_Adult_Site_2012_11_df.shape

(132, 6)

In [168]:
JNorth_Adult_Site_2012_11_df = pd.DataFrame(JNorth_Adult_Site_2012_11_df)
JNorth_Adult_Site_2012_11_df.drop(['month'], axis=1, inplace=True)

C:\Users\motte\anaconda3\envs\mlenv\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [169]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2012_11_df['latitude'] = JNorth_Adult_Site_2012_11_df['latitude'].astype(float)
JNorth_Adult_Site_2012_11_df['longitude'] = JNorth_Adult_Site_2012_11_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

C:\Users\motte\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\motte\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [170]:
geojson = df_to_geojson(JNorth_Adult_Site_2012_11_df, cols)

In [171]:
file = open("JNorth_Adult_Site_2012_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## December

In [172]:
JNorth_Adult_Site_2012_month_df =pd.DataFrame(JNorth_Adult_Site_2012_df)
JNorth_Adult_Site_2012_12_df = JNorth_Adult_Site_2012_month_df[JNorth_Adult_Site_2012_month_df['month']==12]
JNorth_Adult_Site_2012_12_df.tail()

,sighting_date,species,latitude,longitude,number,month
12344,2012-12-26,Monarch Adult Sighted,34.272,-119.231,1,12
12345,2012-12-26,Monarch Adult Sighted,34.423,-119.890,1,12
12346,2012-12-26,Monarch Adult Sighted,35.132,-120.635,1,12
12347,2012-12-26,Monarch Adult Sighted,36.626,-121.930,1,12
12348,2012-12-27,Monarch Adult Sighted,37.063,-122.162,7500,12


In [173]:
JNorth_Adult_Site_2012_12_df.shape

(28, 6)

In [174]:
JNorth_Adult_Site_2012_12_df = pd.DataFrame(JNorth_Adult_Site_2012_12_df)
JNorth_Adult_Site_2012_12_df.drop(['month'], axis=1, inplace=True)

In [175]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2012_12_df['latitude'] = JNorth_Adult_Site_2012_12_df['latitude'].astype(float)
JNorth_Adult_Site_2012_12_df['longitude'] = JNorth_Adult_Site_2012_12_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [176]:
geojson = df_to_geojson(JNorth_Adult_Site_2012_12_df, cols)

In [177]:
file = open("JNorth_Adult_Site_2012_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# 2013

In [178]:
JNorth_Adult_Site_2013_df = JNorth_Adult_Site_year_df[JNorth_Adult_Site_year_df['year']==2013]
JNorth_Adult_Site_2013_df.tail()

,sighting_date,species,latitude,longitude,number,year
16434,2013-12-22,Monarch Adult Sighted,30.034,-99.092,1,2013
16435,2013-12-25,Monarch Adult Sighted,29.821,-98.751,1,2013
16436,2013-12-26,Monarch Adult Sighted,19.590,-100.266,1,2013
16437,2013-12-29,Monarch Adult Sighted,29.757,-95.365,1,2013
16438,2013-12-30,Monarch Adult Sighted,33.398,-117.554,1,2013


In [179]:
JNorth_Adult_Site_2013_df.shape

(4090, 6)

In [180]:
JNorth_Adult_Site_2013_df = pd.DataFrame(JNorth_Adult_Site_2013_df)
JNorth_Adult_Site_2013_df.drop(['year'], axis=1, inplace=True)

In [181]:
JNorth_Adult_Site_2013_df.tail

<bound method NDFrame.tail of       sighting_date                species  latitude  longitude  number
12349    2013-01-01  Monarch Adult Sighted    19.590   -100.266       1
12350    2013-01-01  Monarch Adult Sighted    27.294    -80.350       1
12351    2013-01-01  Monarch Adult Sighted    27.954    -82.457       1
12352    2013-01-01  Monarch Adult Sighted    33.421   -112.069      12
12353    2013-01-01  Monarch Adult Sighted    34.045   -118.682       1
...             ...                    ...       ...        ...     ...
16434    2013-12-22  Monarch Adult Sighted    30.034    -99.092       1
16435    2013-12-25  Monarch Adult Sighted    29.821    -98.751       1
16436    2013-12-26  Monarch Adult Sighted    19.590   -100.266       1
16437    2013-12-29  Monarch Adult Sighted    29.757    -95.365       1
16438    2013-12-30  Monarch Adult Sighted    33.398   -117.554       1

[4090 rows x 5 columns]>

In [182]:
JNorth_Adult_Site_2013_df['month'] = pd.DatetimeIndex(JNorth_Adult_Site_2013_df['sighting_date']).month

## January

In [183]:
JNorth_Adult_Site_2013_month_df =pd.DataFrame(JNorth_Adult_Site_2013_df)
JNorth_Adult_Site_2013_1_df = JNorth_Adult_Site_2013_month_df[JNorth_Adult_Site_2013_month_df['month']==1]
JNorth_Adult_Site_2013_1_df.tail()

,sighting_date,species,latitude,longitude,number,month
12404,2013-01-29,Monarch Adult Sighted,29.524,-95.535,1,1
12405,2013-01-29,Monarch Adult Sighted,33.831,-118.073,1,1
12406,2013-01-31,Monarch Adult Sighted,26.641,-81.872,1,1
12407,2013-01-31,Monarch Adult Sighted,26.990,-82.097,1,1
12408,2013-01-31,Monarch Adult Sighted,29.836,-95.735,1,1


In [184]:
JNorth_Adult_Site_2013_1_df.shape

(60, 6)

In [185]:
JNorth_Adult_Site_2013_1_df = pd.DataFrame(JNorth_Adult_Site_2013_1_df)
JNorth_Adult_Site_2013_1_df.drop(['month'], axis=1, inplace=True)

In [186]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2013_1_df['latitude'] = JNorth_Adult_Site_2013_1_df['latitude'].astype(float)
JNorth_Adult_Site_2013_1_df['longitude'] = JNorth_Adult_Site_2013_1_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [187]:
geojson = df_to_geojson(JNorth_Adult_Site_2013_1_df, cols)

In [188]:
file = open("JNorth_Adult_Site_2013_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## February

In [189]:
JNorth_Adult_Site_2013_month_df =pd.DataFrame(JNorth_Adult_Site_2013_df)
JNorth_Adult_Site_2013_2_df = JNorth_Adult_Site_2013_month_df[JNorth_Adult_Site_2013_month_df['month']==2]
JNorth_Adult_Site_2013_2_df.tail()

,sighting_date,species,latitude,longitude,number,month
12484,2013-02-26,Monarch Adult Sighted,33.212,-95.218,1,2
12485,2013-02-26,Monarch Adult Sighted,33.963,-117.247,1,2
12486,2013-02-27,Monarch Adult Sighted,26.063,-80.233,1,2
12487,2013-02-27,Monarch Adult Sighted,33.761,-118.199,1,2
12488,2013-02-28,Monarch Adult Sighted,35.221,-120.635,1,2


In [190]:
JNorth_Adult_Site_2013_2_df.shape

(80, 6)

In [191]:
JNorth_Adult_Site_2013_2_df = pd.DataFrame(JNorth_Adult_Site_2013_2_df)
JNorth_Adult_Site_2013_2_df.drop(['month'], axis=1, inplace=True)

In [192]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2013_2_df['latitude'] = JNorth_Adult_Site_2013_2_df['latitude'].astype(float)
JNorth_Adult_Site_2013_2_df['longitude'] = JNorth_Adult_Site_2013_2_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [193]:
geojson = df_to_geojson(JNorth_Adult_Site_2013_2_df, cols)

In [194]:
file = open("JNorth_Adult_Site_2013_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## March

In [195]:
JNorth_Adult_Site_2013_month_df =pd.DataFrame(JNorth_Adult_Site_2013_df)
JNorth_Adult_Site_2013_3_df = JNorth_Adult_Site_2013_month_df[JNorth_Adult_Site_2013_month_df['month']==3]
JNorth_Adult_Site_2013_3_df.tail()

,sighting_date,species,latitude,longitude,number,month
12489,2013-03-01,Monarch Adult Sighted,29.635,-95.635,1,3
12490,2013-03-01,Monarch Adult Sighted,29.957,-90.078,1,3
12491,2013-03-09,Monarch Adult Sighted,37.058,-76.464,1,3


In [196]:
JNorth_Adult_Site_2013_3_df.shape

(3, 6)

In [197]:
JNorth_Adult_Site_2013_3_df = pd.DataFrame(JNorth_Adult_Site_2013_3_df)
JNorth_Adult_Site_2013_3_df.drop(['month'], axis=1, inplace=True)

In [198]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2013_3_df['latitude'] = JNorth_Adult_Site_2013_3_df['latitude'].astype(float)
JNorth_Adult_Site_2013_3_df['longitude'] = JNorth_Adult_Site_2013_3_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [199]:
geojson = df_to_geojson(JNorth_Adult_Site_2013_3_df, cols)

In [200]:
file = open("JNorth_Adult_Site_2013_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## April

In [201]:
JNorth_Adult_Site_2013_month_df =pd.DataFrame(JNorth_Adult_Site_2013_df)
JNorth_Adult_Site_2013_4_df = JNorth_Adult_Site_2013_month_df[JNorth_Adult_Site_2013_month_df['month']==4]
JNorth_Adult_Site_2013_4_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [202]:
JNorth_Adult_Site_2013_4_df.shape

(0, 6)

In [203]:
JNorth_Adult_Site_2013_4_df =pd.DataFrame(JNorth_Adult_Site_2013_4_df)
JNorth_Adult_Site_2013_4_df.drop(['month'], axis=1, inplace=True)

In [204]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2013_4_df['latitude'] = JNorth_Adult_Site_2013_4_df['latitude'].astype(float)
JNorth_Adult_Site_2013_4_df['longitude'] = JNorth_Adult_Site_2013_4_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [205]:
geojson = df_to_geojson(JNorth_Adult_Site_2013_4_df, cols)

In [206]:
file = open("JNorth_Adult_Site_2013_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## May

In [207]:
JNorth_Adult_Site_2013_month_df =pd.DataFrame(JNorth_Adult_Site_2013_df)
JNorth_Adult_Site_2013_5_df = JNorth_Adult_Site_2013_month_df[JNorth_Adult_Site_2013_month_df['month']==5]
JNorth_Adult_Site_2013_5_df.tail()

,sighting_date,species,latitude,longitude,number,month
12494,2013-05-15,Monarch Adult Sighted,38.450,-76.220,1,5
12495,2013-05-18,Monarch Adult Sighted,39.746,-75.547,2,5
12496,2013-05-19,Monarch Adult Sighted,32.237,-80.887,1,5
12497,2013-05-25,Monarch Adult Sighted,43.592,-70.369,1,5
12498,2013-05-30,Monarch Adult Sighted,34.604,-106.669,1,5


In [208]:
JNorth_Adult_Site_2013_5_df.shape

(7, 6)

In [209]:
JNorth_Adult_Site_2013_5_df =pd.DataFrame(JNorth_Adult_Site_2013_5_df)
JNorth_Adult_Site_2013_5_df.drop(['month'], axis=1, inplace=True)

In [210]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2013_5_df['latitude'] = JNorth_Adult_Site_2013_5_df['latitude'].astype(float)
JNorth_Adult_Site_2013_5_df['longitude'] = JNorth_Adult_Site_2013_5_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [211]:
geojson = df_to_geojson(JNorth_Adult_Site_2013_5_df, cols)

In [212]:
file = open("JNorth_Adult_Site_2013_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## June

In [213]:
JNorth_Adult_Site_2013_month_df =pd.DataFrame(JNorth_Adult_Site_2013_df)
JNorth_Adult_Site_2013_6_df = JNorth_Adult_Site_2013_month_df[JNorth_Adult_Site_2013_month_df['month']==6]
JNorth_Adult_Site_2013_6_df.tail()

,sighting_date,species,latitude,longitude,number,month
12499,2013-06-15,Monarch Adult Sighted,45.321,-84.445,4,6
12500,2013-06-16,Monarch Adult Sighted,40.783,-123.130,1,6
12501,2013-06-22,Monarch Adult Sighted,40.336,-121.915,1,6
12502,2013-06-25,Monarch Adult Sighted,40.116,-75.014,1,6
12503,2013-06-28,Monarch Adult Sighted,39.763,-86.400,1,6


In [214]:
JNorth_Adult_Site_2013_6_df.shape

(5, 6)

In [215]:
JNorth_Adult_Site_2013_6_df = pd.DataFrame(JNorth_Adult_Site_2013_6_df)
JNorth_Adult_Site_2013_6_df.drop(['month'], axis=1, inplace=True)

In [216]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2013_6_df['latitude'] = JNorth_Adult_Site_2013_6_df['latitude'].astype(float)
JNorth_Adult_Site_2013_6_df['longitude'] = JNorth_Adult_Site_2013_6_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [217]:
geojson = df_to_geojson(JNorth_Adult_Site_2013_6_df, cols)

In [218]:
file = open("JNorth_Adult_Site_2013_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## July

In [219]:
JNorth_Adult_Site_2013_month_df =pd.DataFrame(JNorth_Adult_Site_2013_df)
JNorth_Adult_Site_2013_7_df = JNorth_Adult_Site_2013_month_df[JNorth_Adult_Site_2013_month_df['month']==7]
JNorth_Adult_Site_2013_7_df.tail()

,sighting_date,species,latitude,longitude,number,month
12534,2013-07-31,Monarch Adult Sighted,34.003,-118.028,1,7
12535,2013-07-31,Monarch Adult Sighted,38.445,-93.178,1,7
12536,2013-07-31,Monarch Adult Sighted,40.518,-81.656,1,7
12537,2013-07-31,Monarch Adult Sighted,40.804,-81.990,1,7
12538,2013-07-31,Monarch Adult Sighted,41.118,-76.842,4,7


In [220]:
JNorth_Adult_Site_2013_7_df.shape

(35, 6)

In [221]:
JNorth_Adult_Site_2013_7_df = pd.DataFrame(JNorth_Adult_Site_2013_7_df)
JNorth_Adult_Site_2013_7_df.drop(['month'], axis=1, inplace=True)

In [222]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2013_7_df['latitude'] = JNorth_Adult_Site_2013_7_df['latitude'].astype(float)
JNorth_Adult_Site_2013_7_df['longitude'] = JNorth_Adult_Site_2013_7_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [223]:
geojson = df_to_geojson(JNorth_Adult_Site_2013_7_df, cols)

In [224]:
file = open("JNorth_Adult_Site_2013_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## August

In [225]:
JNorth_Adult_Site_2013_month_df =pd.DataFrame(JNorth_Adult_Site_2013_df)
JNorth_Adult_Site_2013_8_df = JNorth_Adult_Site_2013_month_df[JNorth_Adult_Site_2013_month_df['month']==8]
JNorth_Adult_Site_2013_8_df.tail()

,sighting_date,species,latitude,longitude,number,month
13550,2013-08-31,Monarch Adult Sighted,44.627,-93.240,1,8
13551,2013-08-31,Monarch Adult Sighted,44.738,-91.510,1,8
13552,2013-08-31,Monarch Adult Sighted,44.902,-93.288,8,8
13553,2013-08-31,Monarch Adult Sighted,44.986,-93.270,6,8
13554,2013-08-31,Monarch Adult Sighted,45.317,-77.517,2,8


In [226]:
JNorth_Adult_Site_2013_8_df.shape

(1016, 6)

In [227]:
JNorth_Adult_Site_2013_8_df = pd.DataFrame(JNorth_Adult_Site_2013_8_df)
JNorth_Adult_Site_2013_8_df.drop(['month'], axis=1, inplace=True)

In [228]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2013_8_df['latitude'] = JNorth_Adult_Site_2013_8_df['latitude'].astype(float)
JNorth_Adult_Site_2013_8_df['longitude'] = JNorth_Adult_Site_2013_8_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [229]:
geojson = df_to_geojson(JNorth_Adult_Site_2013_8_df, cols)

In [230]:
file = open("JNorth_Adult_Site_2013_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## September

In [231]:
JNorth_Adult_Site_2013_month_df =pd.DataFrame(JNorth_Adult_Site_2013_df)
JNorth_Adult_Site_2013_9_df = JNorth_Adult_Site_2013_month_df[JNorth_Adult_Site_2013_month_df['month']==9]
JNorth_Adult_Site_2013_9_df.tail()

,sighting_date,species,latitude,longitude,number,month
15093,2013-09-30,Monarch Adult Sighted,44.986,-93.270,2,9
15094,2013-09-30,Monarch Adult Sighted,45.070,-92.890,1,9
15095,2013-09-30,Monarch Adult Sighted,45.187,-87.639,1,9
15096,2013-09-30,Monarch Adult Sighted,45.434,-94.738,1,9
15097,2013-09-30,Monarch Adult Sighted,45.701,-95.023,3,9


In [232]:
JNorth_Adult_Site_2013_9_df.shape

(1543, 6)

In [233]:
JNorth_Adult_Site_2013_9_df = pd.DataFrame(JNorth_Adult_Site_2013_9_df)
JNorth_Adult_Site_2013_9_df.drop(['month'], axis=1, inplace=True)

In [234]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2013_9_df['latitude'] = JNorth_Adult_Site_2013_9_df['latitude'].astype(float)
JNorth_Adult_Site_2013_9_df['longitude'] = JNorth_Adult_Site_2013_9_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [235]:
geojson = df_to_geojson(JNorth_Adult_Site_2013_9_df, cols)

In [236]:
file = open("JNorth_Adult_Site_2013_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## October

In [237]:
JNorth_Adult_Site_2013_month_df =pd.DataFrame(JNorth_Adult_Site_2013_df)
JNorth_Adult_Site_2013_10_df = JNorth_Adult_Site_2013_month_df[JNorth_Adult_Site_2013_month_df['month']==10]
JNorth_Adult_Site_2013_10_df.tail()

,sighting_date,species,latitude,longitude,number,month
16277,2013-10-31,Monarch Adult Sighted,32.398,-97.750,4,10
16278,2013-10-31,Monarch Adult Sighted,32.723,-117.168,5,10
16279,2013-10-31,Monarch Adult Sighted,33.198,-96.615,1,10
16280,2013-10-31,Monarch Adult Sighted,33.768,-78.792,1,10
16281,2013-10-31,Monarch Adult Sighted,38.094,-78.893,4,10


In [238]:
JNorth_Adult_Site_2013_10_df.shape

(1184, 6)

In [239]:
JNorth_Adult_Site_2013_10_df = pd.DataFrame(JNorth_Adult_Site_2013_10_df)
JNorth_Adult_Site_2013_10_df.drop(['month'], axis=1, inplace=True)

In [240]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2013_10_df['latitude'] = JNorth_Adult_Site_2013_10_df['latitude'].astype(float)
JNorth_Adult_Site_2013_10_df['longitude'] = JNorth_Adult_Site_2013_10_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [241]:
geojson = df_to_geojson(JNorth_Adult_Site_2013_10_df, cols)

In [242]:
file = open("JNorth_Adult_Site_2013_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## November

In [243]:
JNorth_Adult_Site_2013_month_df =pd.DataFrame(JNorth_Adult_Site_2013_df)
JNorth_Adult_Site_2013_11_df = JNorth_Adult_Site_2013_month_df[JNorth_Adult_Site_2013_month_df['month']==11]
JNorth_Adult_Site_2013_11_df.tail()

,sighting_date,species,latitude,longitude,number,month
16406,2013-11-25,Monarch Adult Sighted,30.230,-97.714,1,11
16407,2013-11-27,Monarch Adult Sighted,32.797,-106.114,2,11
16408,2013-11-28,Monarch Adult Sighted,30.230,-97.714,1,11
16409,2013-11-29,Monarch Adult Sighted,29.836,-95.735,1,11
16410,2013-11-29,Monarch Adult Sighted,30.930,-97.557,2,11


In [244]:
JNorth_Adult_Site_2013_11_df.shape

(129, 6)

In [245]:
JNorth_Adult_Site_2013_11_df = pd.DataFrame(JNorth_Adult_Site_2013_11_df)
JNorth_Adult_Site_2013_11_df.drop(['month'], axis=1, inplace=True)

In [246]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2013_11_df['latitude'] = JNorth_Adult_Site_2013_11_df['latitude'].astype(float)
JNorth_Adult_Site_2013_11_df['longitude'] = JNorth_Adult_Site_2013_11_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [247]:
geojson = df_to_geojson(JNorth_Adult_Site_2013_11_df, cols)

In [248]:
file = open("JNorth_Adult_Site_2013_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## December

In [249]:
JNorth_Adult_Site_2013_month_df =pd.DataFrame(JNorth_Adult_Site_2013_df)
JNorth_Adult_Site_2013_12_df = JNorth_Adult_Site_2013_month_df[JNorth_Adult_Site_2013_month_df['month']==12]
JNorth_Adult_Site_2013_12_df.tail()

,sighting_date,species,latitude,longitude,number,month
16434,2013-12-22,Monarch Adult Sighted,30.034,-99.092,1,12
16435,2013-12-25,Monarch Adult Sighted,29.821,-98.751,1,12
16436,2013-12-26,Monarch Adult Sighted,19.590,-100.266,1,12
16437,2013-12-29,Monarch Adult Sighted,29.757,-95.365,1,12
16438,2013-12-30,Monarch Adult Sighted,33.398,-117.554,1,12


In [250]:
JNorth_Adult_Site_2013_12_df.shape

(28, 6)

In [251]:
JNorth_Adult_Site_2013_12_df = pd.DataFrame(JNorth_Adult_Site_2013_12_df)
JNorth_Adult_Site_2013_12_df.drop(['month'], axis=1, inplace=True)

In [252]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2013_12_df['latitude'] = JNorth_Adult_Site_2013_12_df['latitude'].astype(float)
JNorth_Adult_Site_2013_12_df['longitude'] = JNorth_Adult_Site_2013_12_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [253]:
geojson = df_to_geojson(JNorth_Adult_Site_2013_12_df, cols)

In [254]:
file = open("JNorth_Adult_Site_2013_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# 2014

In [255]:
JNorth_Adult_Site_2014_df = JNorth_Adult_Site_year_df[JNorth_Adult_Site_year_df['year']==2014]
JNorth_Adult_Site_2014_df.tail()

,sighting_date,species,latitude,longitude,number,year
22638,2014-12-28,Monarch Adult Sighted,33.834,-118.313,3,2014
22639,2014-12-29,Monarch Adult Sighted,30.389,-89.098,10,2014
22640,2014-12-30,Monarch Adult Sighted,34.200,-118.294,2,2014
22641,2014-12-31,Monarch Adult Sighted,28.827,-82.256,1,2014
22642,2014-12-31,Monarch Adult Sighted,33.761,-118.199,1,2014


In [256]:
JNorth_Adult_Site_2014_df.shape

(6204, 6)

In [257]:
JNorth_Adult_Site_2014_df = pd.DataFrame(JNorth_Adult_Site_2014_df)
JNorth_Adult_Site_2014_df.drop(['year'], axis=1, inplace=True)

In [258]:
JNorth_Adult_Site_2014_df.tail

<bound method NDFrame.tail of       sighting_date                species  latitude  longitude  number
16439    2014-01-01  Monarch Adult Sighted    19.590   -100.266       1
16440    2014-01-01  Monarch Adult Sighted    19.670   -100.280       1
16441    2014-01-01  Monarch Adult Sighted    28.984    -81.057       1
16442    2014-01-01  Monarch Adult Sighted    33.373   -112.390       1
16443    2014-01-01  Monarch Adult Sighted    34.559   -120.545   11082
...             ...                    ...       ...        ...     ...
22638    2014-12-28  Monarch Adult Sighted    33.834   -118.313       3
22639    2014-12-29  Monarch Adult Sighted    30.389    -89.098      10
22640    2014-12-30  Monarch Adult Sighted    34.200   -118.294       2
22641    2014-12-31  Monarch Adult Sighted    28.827    -82.256       1
22642    2014-12-31  Monarch Adult Sighted    33.761   -118.199       1

[6204 rows x 5 columns]>

In [259]:
JNorth_Adult_Site_2014_df['month'] = pd.DatetimeIndex(JNorth_Adult_Site_2014_df['sighting_date']).month

## January

In [260]:
JNorth_Adult_Site_2014_month_df =pd.DataFrame(JNorth_Adult_Site_2014_df)
JNorth_Adult_Site_2014_1_df = JNorth_Adult_Site_2014_month_df[JNorth_Adult_Site_2014_month_df['month']==1]
JNorth_Adult_Site_2014_1_df.tail()

,sighting_date,species,latitude,longitude,number,month
16488,2014-01-28,Monarch Adult Sighted,37.777,-122.217,1,1
16489,2014-01-29,Monarch Adult Sighted,27.771,-82.679,1,1
16490,2014-01-30,Monarch Adult Sighted,32.723,-117.168,1,1
16491,2014-01-30,Monarch Adult Sighted,33.212,-117.357,1,1
16492,2014-01-30,Monarch Adult Sighted,34.097,-117.906,1,1


In [261]:
JNorth_Adult_Site_2014_1_df.shape

(54, 6)

In [262]:
JNorth_Adult_Site_2014_1_df = pd.DataFrame(JNorth_Adult_Site_2014_1_df)
JNorth_Adult_Site_2014_1_df.drop(['month'], axis=1, inplace=True)

In [263]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2014_1_df['latitude'] = JNorth_Adult_Site_2014_1_df['latitude'].astype(float)
JNorth_Adult_Site_2014_1_df['longitude'] = JNorth_Adult_Site_2014_1_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [264]:
geojson = df_to_geojson(JNorth_Adult_Site_2014_1_df, cols)

In [265]:
file = open("JNorth_Adult_Site_2014_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## February

In [266]:
JNorth_Adult_Site_2014_month_df =pd.DataFrame(JNorth_Adult_Site_2014_df)
JNorth_Adult_Site_2014_2_df = JNorth_Adult_Site_2014_month_df[JNorth_Adult_Site_2014_month_df['month']==2]
JNorth_Adult_Site_2014_2_df.tail()

,sighting_date,species,latitude,longitude,number,month
16523,2014-02-20,Monarch Adult Sighted,38.757,-123.512,1,2
16524,2014-02-20,Monarch Adult Sighted,38.832,-123.436,1,2
16525,2014-02-23,Monarch Adult Sighted,27.915,-82.802,1,2
16526,2014-02-23,Monarch Adult Sighted,29.757,-95.365,1,2
16527,2014-02-24,Monarch Adult Sighted,27.337,-82.514,1,2


In [267]:
JNorth_Adult_Site_2014_2_df.shape

(35, 6)

In [268]:
JNorth_Adult_Site_2014_2_df = pd.DataFrame(JNorth_Adult_Site_2014_2_df)
JNorth_Adult_Site_2014_2_df.drop(['month'], axis=1, inplace=True)

In [269]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2014_2_df['latitude'] = JNorth_Adult_Site_2014_2_df['latitude'].astype(float)
JNorth_Adult_Site_2014_2_df['longitude'] = JNorth_Adult_Site_2014_2_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [270]:
geojson = df_to_geojson(JNorth_Adult_Site_2014_2_df, cols)

In [271]:
file = open("JNorth_Adult_Site_2014_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## March

In [272]:
JNorth_Adult_Site_2014_month_df =pd.DataFrame(JNorth_Adult_Site_2014_df)
JNorth_Adult_Site_2014_3_df = JNorth_Adult_Site_2014_month_df[JNorth_Adult_Site_2014_month_df['month']==3]
JNorth_Adult_Site_2014_3_df.tail()

,sighting_date,species,latitude,longitude,number,month
16528,2014-03-01,Monarch Adult Sighted,29.757,-95.365,1,3


In [273]:
JNorth_Adult_Site_2014_3_df.shape

(1, 6)

In [274]:
JNorth_Adult_Site_2014_3_df = pd.DataFrame(JNorth_Adult_Site_2014_3_df)
JNorth_Adult_Site_2014_3_df.drop(['month'], axis=1, inplace=True)

In [275]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2014_3_df['latitude'] = JNorth_Adult_Site_2014_3_df['latitude'].astype(float)
JNorth_Adult_Site_2014_3_df['longitude'] = JNorth_Adult_Site_2014_3_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [276]:
geojson = df_to_geojson(JNorth_Adult_Site_2014_3_df, cols)

In [277]:
file = open("JNorth_Adult_Site_2014_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## April

In [278]:
JNorth_Adult_Site_2014_month_df =pd.DataFrame(JNorth_Adult_Site_2014_df)
JNorth_Adult_Site_2014_4_df = JNorth_Adult_Site_2014_month_df[JNorth_Adult_Site_2014_month_df['month']==4]
JNorth_Adult_Site_2014_4_df.tail()

,sighting_date,species,latitude,longitude,number,month
16529,2014-04-06,Monarch Adult Sighted,28.542,-81.374,1,4
16530,2014-04-09,Monarch Adult Sighted,44.292,-74.326,6,4
16531,2014-04-09,Monarch Adult Sighted,44.300,-79.367,1,4


In [279]:
JNorth_Adult_Site_2014_4_df.shape

(3, 6)

In [280]:
JNorth_Adult_Site_2014_4_df = pd.DataFrame(JNorth_Adult_Site_2014_4_df)
JNorth_Adult_Site_2014_4_df.drop(['month'], axis=1, inplace=True)

In [281]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2014_4_df['latitude'] = JNorth_Adult_Site_2014_4_df['latitude'].astype(float)
JNorth_Adult_Site_2014_4_df['longitude'] = JNorth_Adult_Site_2014_4_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [282]:
geojson = df_to_geojson(JNorth_Adult_Site_2014_4_df, cols)

In [283]:
file = open("JNorth_Adult_Site_2014_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## May

In [284]:
JNorth_Adult_Site_2014_month_df =pd.DataFrame(JNorth_Adult_Site_2014_df)
JNorth_Adult_Site_2014_5_df = JNorth_Adult_Site_2014_month_df[JNorth_Adult_Site_2014_month_df['month']==5]
JNorth_Adult_Site_2014_5_df.tail()

,sighting_date,species,latitude,longitude,number,month
16535,2014-05-19,Monarch Adult Sighted,39.104,-94.601,1,5
16536,2014-05-22,Monarch Adult Sighted,43.633,-84.788,1,5
16537,2014-05-24,Monarch Adult Sighted,38.631,-90.192,2,5
16538,2014-05-26,Monarch Adult Sighted,44.753,-85.701,1,5
16539,2014-05-30,Monarch Adult Sighted,41.886,-87.623,1,5


In [285]:
JNorth_Adult_Site_2014_5_df.shape

(8, 6)

In [286]:
JNorth_Adult_Site_2014_5_df = pd.DataFrame(JNorth_Adult_Site_2014_5_df)
JNorth_Adult_Site_2014_5_df.drop(['month'], axis=1, inplace=True)

In [287]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2014_5_df['latitude'] = JNorth_Adult_Site_2014_5_df['latitude'].astype(float)
JNorth_Adult_Site_2014_5_df['longitude'] = JNorth_Adult_Site_2014_5_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [288]:
geojson = df_to_geojson(JNorth_Adult_Site_2014_5_df, cols)

In [289]:
file = open("JNorth_Adult_Site_2014_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## June

In [290]:
JNorth_Adult_Site_2014_month_df =pd.DataFrame(JNorth_Adult_Site_2014_df)
JNorth_Adult_Site_2014_6_df = JNorth_Adult_Site_2014_month_df[JNorth_Adult_Site_2014_month_df['month']==6]
JNorth_Adult_Site_2014_6_df.tail()

,sighting_date,species,latitude,longitude,number,month
16589,2014-06-30,Monarch Adult Sighted,42.988,-89.570,1,6
16590,2014-06-30,Monarch Adult Sighted,44.305,-77.800,1,6
16591,2014-06-30,Monarch Adult Sighted,44.350,-78.733,1,6
16592,2014-06-30,Monarch Adult Sighted,45.050,-78.517,2,6
16593,2014-06-30,Monarch Adult Sighted,45.427,-84.537,1,6


In [291]:
JNorth_Adult_Site_2014_6_df.shape

(54, 6)

In [292]:
JNorth_Adult_Site_2014_6_df = pd.DataFrame(JNorth_Adult_Site_2014_6_df)
JNorth_Adult_Site_2014_6_df.drop(['month'], axis=1, inplace=True)

In [293]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2014_6_df['latitude'] = JNorth_Adult_Site_2014_6_df['latitude'].astype(float)
JNorth_Adult_Site_2014_6_df['longitude'] = JNorth_Adult_Site_2014_6_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [294]:
geojson = df_to_geojson(JNorth_Adult_Site_2014_6_df, cols)

In [295]:
file = open("JNorth_Adult_Site_2014_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## July

In [296]:
JNorth_Adult_Site_2014_month_df =pd.DataFrame(JNorth_Adult_Site_2014_df)
JNorth_Adult_Site_2014_7_df = JNorth_Adult_Site_2014_month_df[JNorth_Adult_Site_2014_month_df['month']==7]
JNorth_Adult_Site_2014_7_df.tail()

,sighting_date,species,latitude,longitude,number,month
17389,2014-07-31,Monarch Adult Sighted,43.183,-79.233,1,7
17390,2014-07-31,Monarch Adult Sighted,43.633,-79.967,1,7
17391,2014-07-31,Monarch Adult Sighted,43.650,-79.917,1,7
17392,2014-07-31,Monarch Adult Sighted,43.700,-79.417,2,7
17393,2014-07-31,Monarch Adult Sighted,43.903,-69.980,2,7


In [297]:
JNorth_Adult_Site_2014_7_df.shape

(800, 6)

In [298]:
JNorth_Adult_Site_2014_7_df = pd.DataFrame(JNorth_Adult_Site_2014_7_df)
JNorth_Adult_Site_2014_7_df.drop(['month'], axis=1, inplace=True)

In [299]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2014_7_df['latitude'] = JNorth_Adult_Site_2014_7_df['latitude'].astype(float)
JNorth_Adult_Site_2014_7_df['longitude'] = JNorth_Adult_Site_2014_7_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [300]:
geojson = df_to_geojson(JNorth_Adult_Site_2014_7_df, cols)

In [301]:
file = open("JNorth_Adult_Site_2014_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## August

In [302]:
JNorth_Adult_Site_2014_month_df =pd.DataFrame(JNorth_Adult_Site_2014_df)
JNorth_Adult_Site_2014_8_df = JNorth_Adult_Site_2014_month_df[JNorth_Adult_Site_2014_month_df['month']==8]
JNorth_Adult_Site_2014_8_df.tail()

,sighting_date,species,latitude,longitude,number,month
18631,2014-08-31,Monarch Adult Sighted,45.104,-83.460,5,8
18632,2014-08-31,Monarch Adult Sighted,45.170,-93.390,1,8
18633,2014-08-31,Monarch Adult Sighted,46.113,-74.477,1,8
18634,2014-08-31,Monarch Adult Sighted,46.154,-85.338,1,8
18635,2014-08-31,Monarch Adult Sighted,46.158,-89.412,10,8


In [303]:
JNorth_Adult_Site_2014_8_df.shape

(1242, 6)

In [304]:
JNorth_Adult_Site_2014_8_df= pd.DataFrame(JNorth_Adult_Site_2014_8_df)
JNorth_Adult_Site_2014_8_df.drop(['month'], axis=1, inplace=True)

In [305]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2014_8_df['latitude'] = JNorth_Adult_Site_2014_8_df['latitude'].astype(float)
JNorth_Adult_Site_2014_8_df['longitude'] = JNorth_Adult_Site_2014_8_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [306]:
geojson = df_to_geojson(JNorth_Adult_Site_2014_8_df, cols)

In [307]:
file = open("JNorth_Adult_Site_2014_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## September

In [308]:
JNorth_Adult_Site_2014_month_df =pd.DataFrame(JNorth_Adult_Site_2014_df)
JNorth_Adult_Site_2014_9_df = JNorth_Adult_Site_2014_month_df[JNorth_Adult_Site_2014_month_df['month']==9]
JNorth_Adult_Site_2014_9_df.tail()

,sighting_date,species,latitude,longitude,number,month
20724,2014-09-30,Monarch Adult Sighted,43.750,-81.717,1,9
20725,2014-09-30,Monarch Adult Sighted,43.771,-72.787,1,9
20726,2014-09-30,Monarch Adult Sighted,43.900,-78.850,20,9
20727,2014-09-30,Monarch Adult Sighted,44.648,-83.361,3,9
20728,2014-09-30,Monarch Adult Sighted,49.884,-97.141,1,9


In [309]:
JNorth_Adult_Site_2014_9_df.shape

(2093, 6)

In [310]:
JNorth_Adult_Site_2014_9_df = pd.DataFrame(JNorth_Adult_Site_2014_9_df)
JNorth_Adult_Site_2014_9_df.drop(['month'], axis=1, inplace=True)

In [311]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2014_9_df['latitude'] = JNorth_Adult_Site_2014_9_df['latitude'].astype(float)
JNorth_Adult_Site_2014_9_df['longitude'] = JNorth_Adult_Site_2014_9_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [312]:
geojson = df_to_geojson(JNorth_Adult_Site_2014_9_df, cols)

In [313]:
file = open("JNorth_Adult_Site_2014_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## October

In [314]:
JNorth_Adult_Site_2014_month_df =pd.DataFrame(JNorth_Adult_Site_2014_df)
JNorth_Adult_Site_2014_10_df = JNorth_Adult_Site_2014_month_df[JNorth_Adult_Site_2014_month_df['month']==10]
JNorth_Adult_Site_2014_10_df.tail()

,sighting_date,species,latitude,longitude,number,month
22452,2014-10-31,Monarch Adult Sighted,29.757,-95.365,1,10
22453,2014-10-31,Monarch Adult Sighted,30.230,-97.714,6,10
22454,2014-10-31,Monarch Adult Sighted,35.257,-90.807,1,10
22455,2014-10-31,Monarch Adult Sighted,38.262,-77.121,2,10
22456,2014-10-31,Monarch Adult Sighted,39.294,-82.082,1,10


In [315]:
JNorth_Adult_Site_2014_10_df.shape

(1728, 6)

In [316]:
JNorth_Adult_Site_2014_10_df = pd.DataFrame(JNorth_Adult_Site_2014_10_df)
JNorth_Adult_Site_2014_10_df.drop(['month'], axis=1, inplace=True)

In [317]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2014_10_df['latitude'] = JNorth_Adult_Site_2014_10_df['latitude'].astype(float)
JNorth_Adult_Site_2014_10_df['longitude'] = JNorth_Adult_Site_2014_10_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [318]:
geojson = df_to_geojson(JNorth_Adult_Site_2014_10_df, cols)

In [319]:
file = open("JNorth_Adult_Site_2014_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## November

In [320]:
JNorth_Adult_Site_2014_month_df =pd.DataFrame(JNorth_Adult_Site_2014_df)
JNorth_Adult_Site_2014_11_df = JNorth_Adult_Site_2014_month_df[JNorth_Adult_Site_2014_month_df['month']==11]
JNorth_Adult_Site_2014_11_df.tail()

,sighting_date,species,latitude,longitude,number,month
22594,2014-11-28,Monarch Adult Sighted,30.230,-97.714,2,11
22595,2014-11-28,Monarch Adult Sighted,30.410,-86.760,6,11
22596,2014-11-29,Monarch Adult Sighted,25.666,-100.215,1,11
22597,2014-11-29,Monarch Adult Sighted,34.442,-119.863,3,11
22598,2014-11-30,Monarch Adult Sighted,29.757,-95.365,2,11


In [321]:
JNorth_Adult_Site_2014_11_df.shape

(142, 6)

In [322]:
JNorth_Adult_Site_2014_11_df = pd.DataFrame(JNorth_Adult_Site_2014_11_df)
JNorth_Adult_Site_2014_11_df.drop(['month'], axis=1, inplace=True)

In [323]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2014_11_df['latitude'] = JNorth_Adult_Site_2014_11_df['latitude'].astype(float)
JNorth_Adult_Site_2014_11_df['longitude'] = JNorth_Adult_Site_2014_11_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [324]:
geojson = df_to_geojson(JNorth_Adult_Site_2014_11_df, cols)

In [325]:
file = open("JNorth_Adult_Site_2014_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## December

In [326]:
JNorth_Adult_Site_2014_month_df =pd.DataFrame(JNorth_Adult_Site_2014_df)
JNorth_Adult_Site_2014_12_df = JNorth_Adult_Site_2014_month_df[JNorth_Adult_Site_2014_month_df['month']==12]
JNorth_Adult_Site_2014_12_df.tail()

,sighting_date,species,latitude,longitude,number,month
22638,2014-12-28,Monarch Adult Sighted,33.834,-118.313,3,12
22639,2014-12-29,Monarch Adult Sighted,30.389,-89.098,10,12
22640,2014-12-30,Monarch Adult Sighted,34.200,-118.294,2,12
22641,2014-12-31,Monarch Adult Sighted,28.827,-82.256,1,12
22642,2014-12-31,Monarch Adult Sighted,33.761,-118.199,1,12


In [327]:
JNorth_Adult_Site_2014_12_df.shape

(44, 6)

In [328]:
JNorth_Adult_Site_2014_12_df = pd.DataFrame(JNorth_Adult_Site_2014_12_df)
JNorth_Adult_Site_2014_12_df.drop(['month'], axis=1, inplace=True)

In [329]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2014_12_df['latitude'] = JNorth_Adult_Site_2014_12_df['latitude'].astype(float)
JNorth_Adult_Site_2014_12_df['longitude'] = JNorth_Adult_Site_2014_12_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [330]:
geojson = df_to_geojson(JNorth_Adult_Site_2014_12_df, cols)

In [331]:
file = open("JNorth_Adult_Site_2014_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# 2015

In [332]:
JNorth_Adult_Site_2015_df = JNorth_Adult_Site_df[JNorth_Adult_Site_df['year']==2015]
JNorth_Adult_Site_2015_df.tail()

,sighting_date,species,latitude,longitude,number,year
30819,2015-12-26,Monarch Adult Sighted,19.590,-100.266,1,2015
30820,2015-12-26,Monarch Adult Sighted,30.047,-95.214,1,2015
30821,2015-12-27,Monarch Adult Sighted,29.272,-95.729,1,2015
30822,2015-12-27,Monarch Adult Sighted,32.237,-80.887,1,2015
30823,2015-12-27,Monarch Adult Sighted,32.237,-80.860,1,2015


In [333]:
JNorth_Adult_Site_2015_df.shape

(8181, 6)

In [334]:
JNorth_Adult_Site_2015_df = pd.DataFrame(JNorth_Adult_Site_2015_df)
JNorth_Adult_Site_2015_df.drop(['year'], axis=1, inplace=True)

In [335]:
JNorth_Adult_Site_2015_df.tail

<bound method NDFrame.tail of       sighting_date                species  latitude  longitude  number
22643    2015-01-01  Monarch Adult Sighted    19.590   -100.266       1
22644    2015-01-01  Monarch Adult Sighted    19.670   -100.280       1
22645    2015-01-01  Monarch Adult Sighted    26.978    -82.199       1
22646    2015-01-01  Monarch Adult Sighted    27.770    -82.637       2
22647    2015-01-01  Monarch Adult Sighted    34.346   -119.329       2
...             ...                    ...       ...        ...     ...
30819    2015-12-26  Monarch Adult Sighted    19.590   -100.266       1
30820    2015-12-26  Monarch Adult Sighted    30.047    -95.214       1
30821    2015-12-27  Monarch Adult Sighted    29.272    -95.729       1
30822    2015-12-27  Monarch Adult Sighted    32.237    -80.887       1
30823    2015-12-27  Monarch Adult Sighted    32.237    -80.860       1

[8181 rows x 5 columns]>

In [336]:
JNorth_Adult_Site_2015_df['month'] = pd.DatetimeIndex(JNorth_Adult_Site_2015_df['sighting_date']).month

## January

In [337]:
JNorth_Adult_Site_2015_month_df =pd.DataFrame(JNorth_Adult_Site_2015_df)
JNorth_Adult_Site_2015_1_df = JNorth_Adult_Site_2015_month_df[JNorth_Adult_Site_2015_month_df['month']==1]
JNorth_Adult_Site_2015_1_df.tail()

,sighting_date,species,latitude,longitude,number,month
22709,2015-01-31,Monarch Adult Sighted,28.529,-96.679,1,1
22710,2015-01-31,Monarch Adult Sighted,29.750,-85.294,1,1
22711,2015-01-31,Monarch Adult Sighted,33.761,-118.199,1,1
22712,2015-01-31,Monarch Adult Sighted,37.378,-122.020,1,1
22713,2015-01-31,Monarch Adult Sighted,37.417,-122.048,1,1


In [338]:
JNorth_Adult_Site_2015_1_df.shape

(71, 6)

In [339]:
JNorth_Adult_Site_2015_1_df = pd.DataFrame(JNorth_Adult_Site_2015_1_df)
JNorth_Adult_Site_2015_1_df.drop(['month'], axis=1, inplace=True)

In [340]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2015_1_df['latitude'] = JNorth_Adult_Site_2015_1_df['latitude'].astype(float)
JNorth_Adult_Site_2015_1_df['longitude'] = JNorth_Adult_Site_2015_1_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [341]:
geojson = df_to_geojson(JNorth_Adult_Site_2015_1_df, cols)

In [342]:
file = open("JNorth_Adult_Site_2015_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# February

In [343]:
JNorth_Adult_Site_2015_month_df =pd.DataFrame(JNorth_Adult_Site_2015_df)
JNorth_Adult_Site_2015_2_df = JNorth_Adult_Site_2015_month_df[JNorth_Adult_Site_2015_month_df['month']==2]
JNorth_Adult_Site_2015_2_df.tail()

,sighting_date,species,latitude,longitude,number,month
22762,2015-02-21,Monarch Adult Sighted,29.957,-90.078,1,2
22763,2015-02-22,Monarch Adult Sighted,28.029,-80.608,1,2
22764,2015-02-22,Monarch Adult Sighted,29.469,-98.528,1,2
22765,2015-02-22,Monarch Adult Sighted,31.566,-81.444,1,2
22766,2015-02-24,Monarch Adult Sighted,28.711,-81.347,1,2


In [344]:
JNorth_Adult_Site_2015_2_df.shape

(53, 6)

In [345]:
JNorth_Adult_Site_2015_2_df = pd.DataFrame(JNorth_Adult_Site_2015_2_df)
JNorth_Adult_Site_2015_2_df.drop(['month'], axis=1, inplace=True)

In [346]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2015_2_df['latitude'] = JNorth_Adult_Site_2015_2_df['latitude'].astype(float)
JNorth_Adult_Site_2015_2_df['longitude'] = JNorth_Adult_Site_2015_2_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [347]:
geojson = df_to_geojson(JNorth_Adult_Site_2015_2_df, cols)

In [348]:
file = open("JNorth_Adult_Site_2015_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# March

In [349]:
JNorth_Adult_Site_2015_month_df =pd.DataFrame(JNorth_Adult_Site_2015_df)
JNorth_Adult_Site_2015_3_df = JNorth_Adult_Site_2015_month_df[JNorth_Adult_Site_2015_month_df['month']==3]
JNorth_Adult_Site_2015_3_df.tail()

,sighting_date,species,latitude,longitude,number,month
22776,2015-03-08,Monarch Adult Sighted,32.723,-117.168,1,3
22777,2015-03-08,Monarch Adult Sighted,44.633,-75.950,5,3
22778,2015-03-08,Monarch Adult Sighted,44.883,-65.167,1,3
22779,2015-03-09,Monarch Adult Sighted,30.455,-84.253,1,3
22780,2015-03-15,Monarch Adult Sighted,25.942,-81.718,1,3


In [350]:
JNorth_Adult_Site_2015_3_df.shape

(14, 6)

In [351]:
JNorth_Adult_Site_2015_3_df = pd.DataFrame(JNorth_Adult_Site_2015_3_df)
JNorth_Adult_Site_2015_3_df.drop(['month'], axis=1, inplace=True)

In [352]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2015_3_df['latitude'] = JNorth_Adult_Site_2015_3_df['latitude'].astype(float)
JNorth_Adult_Site_2015_3_df['longitude'] = JNorth_Adult_Site_2015_3_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [353]:
geojson = df_to_geojson(JNorth_Adult_Site_2015_3_df, cols)

In [354]:
file = open("JNorth_Adult_Site_2015_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# April

In [355]:
JNorth_Adult_Site_2015_month_df =pd.DataFrame(JNorth_Adult_Site_2015_df)
JNorth_Adult_Site_2015_4_df = JNorth_Adult_Site_2015_month_df[JNorth_Adult_Site_2015_month_df['month']==4]
JNorth_Adult_Site_2015_4_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [356]:
JNorth_Adult_Site_2015_4_df.shape

(0, 6)

In [357]:
JNorth_Adult_Site_2015_4_df= pd.DataFrame(JNorth_Adult_Site_2015_4_df)
JNorth_Adult_Site_2015_4_df.drop(['month'], axis=1, inplace=True)

In [358]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2015_4_df['latitude'] = JNorth_Adult_Site_2015_4_df['latitude'].astype(float)
JNorth_Adult_Site_2015_4_df['longitude'] = JNorth_Adult_Site_2015_4_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [359]:
geojson = df_to_geojson(JNorth_Adult_Site_2015_4_df, cols)

In [360]:
file = open("JNorth_Adult_Site_2015_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# May

In [367]:
JNorth_Adult_Site_2015_month_df =pd.DataFrame(JNorth_Adult_Site_2015_df)
JNorth_Adult_Site_2015_5_df = JNorth_Adult_Site_2015_month_df[JNorth_Adult_Site_2015_month_df['month']==5]
JNorth_Adult_Site_2015_5_df.tail()

,sighting_date,species,latitude,longitude,number,month
22782,2015-05-04,Monarch Adult Sighted,34.413,-119.861,24,5
22783,2015-05-04,Monarch Adult Sighted,40.852,-91.097,2,5
22784,2015-05-08,Monarch Adult Sighted,46.183,-82.955,4,5
22785,2015-05-15,Monarch Adult Sighted,44.950,-76.333,1,5
22786,2015-05-28,Monarch Adult Sighted,29.757,-95.365,1,5


In [368]:
JNorth_Adult_Site_2015_5_df.shape

(6, 6)

In [369]:
JNorth_Adult_Site_2015_5_df = pd.DataFrame(JNorth_Adult_Site_2015_5_df)
JNorth_Adult_Site_2015_5_df.drop(['month'], axis=1, inplace=True)

In [370]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2015_5_df['latitude'] = JNorth_Adult_Site_2015_5_df['latitude'].astype(float)
JNorth_Adult_Site_2015_5_df['longitude'] = JNorth_Adult_Site_2015_5_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [371]:
geojson = df_to_geojson(JNorth_Adult_Site_2015_5_df, cols)

In [372]:
file = open("JNorth_Adult_Site_2015_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# June

In [373]:
JNorth_Adult_Site_2015_month_df =pd.DataFrame(JNorth_Adult_Site_2015_df)
JNorth_Adult_Site_2015_6_df = JNorth_Adult_Site_2015_month_df[JNorth_Adult_Site_2015_month_df['month']==6]
JNorth_Adult_Site_2015_6_df.tail()

,sighting_date,species,latitude,longitude,number,month
22802,2015-06-28,Monarch Adult Sighted,49.188,-98.100,2,6
22803,2015-06-29,Monarch Adult Sighted,39.586,-79.959,2,6
22804,2015-06-29,Monarch Adult Sighted,43.450,-80.483,1,6
22805,2015-06-30,Monarch Adult Sighted,41.760,-72.959,1,6
22806,2015-06-30,Monarch Adult Sighted,42.983,-81.233,1,6


In [374]:
JNorth_Adult_Site_2015_6_df.shape

(20, 6)

In [375]:
JNorth_Adult_Site_2015_6_df = pd.DataFrame(JNorth_Adult_Site_2015_6_df)
JNorth_Adult_Site_2015_6_df.drop(['month'], axis=1, inplace=True)

In [376]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2015_6_df['latitude'] = JNorth_Adult_Site_2015_6_df['latitude'].astype(float)
JNorth_Adult_Site_2015_6_df['longitude'] = JNorth_Adult_Site_2015_6_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [377]:
geojson = df_to_geojson(JNorth_Adult_Site_2015_6_df, cols)

In [378]:
file = open("JNorth_Adult_Site_2015_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# July

In [379]:
JNorth_Adult_Site_2015_month_df =pd.DataFrame(JNorth_Adult_Site_2015_df)
JNorth_Adult_Site_2015_7_df = JNorth_Adult_Site_2015_month_df[JNorth_Adult_Site_2015_month_df['month']==7]
JNorth_Adult_Site_2015_7_df.tail()

,sighting_date,species,latitude,longitude,number,month
23207,2015-07-31,Monarch Adult Sighted,44.683,-76.400,5,7
23208,2015-07-31,Monarch Adult Sighted,44.735,-79.769,1,7
23209,2015-07-31,Monarch Adult Sighted,44.808,-85.244,1,7
23210,2015-07-31,Monarch Adult Sighted,45.030,-93.320,3,7
23211,2015-07-31,Monarch Adult Sighted,47.665,-117.435,1,7


In [380]:
JNorth_Adult_Site_2015_7_df.shape

(405, 6)

In [381]:
JNorth_Adult_Site_2015_7_df = pd.DataFrame(JNorth_Adult_Site_2015_7_df)
JNorth_Adult_Site_2015_7_df.drop(['month'], axis=1, inplace=True)

In [382]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2015_7_df['latitude'] = JNorth_Adult_Site_2015_7_df['latitude'].astype(float)
JNorth_Adult_Site_2015_7_df['longitude'] = JNorth_Adult_Site_2015_7_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [383]:
geojson = df_to_geojson(JNorth_Adult_Site_2015_7_df, cols)

In [384]:
file = open("JNorth_Adult_Site_2015_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# August

In [385]:
JNorth_Adult_Site_2015_month_df =pd.DataFrame(JNorth_Adult_Site_2015_df)
JNorth_Adult_Site_2015_8_df = JNorth_Adult_Site_2015_month_df[JNorth_Adult_Site_2015_month_df['month']==8]
JNorth_Adult_Site_2015_8_df.tail()

,sighting_date,species,latitude,longitude,number,month
25256,2015-08-31,Monarch Adult Sighted,46.317,-79.467,2,8
25257,2015-08-31,Monarch Adult Sighted,46.383,-82.633,1,8
25258,2015-08-31,Monarch Adult Sighted,48.239,-96.496,5,8
25259,2015-08-31,Monarch Adult Sighted,49.817,-94.433,2,8
25260,2015-08-31,Monarch Adult Sighted,49.884,-97.141,1,8


In [386]:
JNorth_Adult_Site_2015_8_df.shape

(2049, 6)

In [387]:
JNorth_Adult_Site_2015_8_df = pd.DataFrame(JNorth_Adult_Site_2015_8_df)
JNorth_Adult_Site_2015_8_df.drop(['month'], axis=1, inplace=True)

In [388]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2015_8_df['latitude'] = JNorth_Adult_Site_2015_8_df['latitude'].astype(float)
JNorth_Adult_Site_2015_8_df['longitude'] = JNorth_Adult_Site_2015_8_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [389]:
geojson = df_to_geojson(JNorth_Adult_Site_2015_8_df, cols)

In [390]:
file = open("JNorth_Adult_Site_2015_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# September

In [391]:
JNorth_Adult_Site_2015_month_df =pd.DataFrame(JNorth_Adult_Site_2015_df)
JNorth_Adult_Site_2015_9_df = JNorth_Adult_Site_2015_month_df[JNorth_Adult_Site_2015_month_df['month']==9]
JNorth_Adult_Site_2015_9_df.tail()

,sighting_date,species,latitude,longitude,number,month
28502,2015-09-30,Monarch Adult Sighted,42.400,-82.183,1,9
28503,2015-09-30,Monarch Adult Sighted,43.367,-80.950,1,9
28504,2015-09-30,Monarch Adult Sighted,43.731,-73.559,1,9
28505,2015-09-30,Monarch Adult Sighted,45.020,-93.218,2,9
28506,2015-09-30,Monarch Adult Sighted,47.112,-91.857,1,9


In [392]:
JNorth_Adult_Site_2015_9_df.shape

(3246, 6)

In [393]:
JNorth_Adult_Site_2015_9_df = pd.DataFrame(JNorth_Adult_Site_2015_9_df)
JNorth_Adult_Site_2015_9_df.drop(['month'], axis=1, inplace=True)

In [394]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2015_9_df['latitude'] = JNorth_Adult_Site_2015_9_df['latitude'].astype(float)
JNorth_Adult_Site_2015_9_df['longitude'] = JNorth_Adult_Site_2015_9_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [395]:
geojson = df_to_geojson(JNorth_Adult_Site_2015_9_df, cols)

In [396]:
file = open("JNorth_Adult_Site_2015_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# October

In [397]:
JNorth_Adult_Site_2015_month_df =pd.DataFrame(JNorth_Adult_Site_2015_df)
JNorth_Adult_Site_2015_10_df = JNorth_Adult_Site_2015_month_df[JNorth_Adult_Site_2015_month_df['month']==10]
JNorth_Adult_Site_2015_10_df.tail()

,sighting_date,species,latitude,longitude,number,month
30329,2015-10-31,Monarch Adult Sighted,36.846,-76.092,3,10
30330,2015-10-31,Monarch Adult Sighted,38.482,-122.435,1,10
30331,2015-10-31,Monarch Adult Sighted,38.968,-74.924,2,10
30332,2015-10-31,Monarch Adult Sighted,39.097,-84.139,1,10
30333,2015-10-31,Monarch Adult Sighted,40.821,-73.924,1,10


In [398]:
JNorth_Adult_Site_2015_10_df.shape

(1827, 6)

In [399]:
JNorth_Adult_Site_2015_10_df = pd.DataFrame(JNorth_Adult_Site_2015_10_df)
JNorth_Adult_Site_2015_10_df.drop(['month'], axis=1, inplace=True)

In [400]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2015_10_df['latitude'] = JNorth_Adult_Site_2015_10_df['latitude'].astype(float)
JNorth_Adult_Site_2015_10_df['longitude'] = JNorth_Adult_Site_2015_10_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [401]:
geojson = df_to_geojson(JNorth_Adult_Site_2015_10_df, cols)

In [402]:
file = open("JNorth_Adult_Site_2015_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# November

In [403]:
JNorth_Adult_Site_2015_month_df =pd.DataFrame(JNorth_Adult_Site_2015_df)
JNorth_Adult_Site_2015_11_df = JNorth_Adult_Site_2015_month_df[JNorth_Adult_Site_2015_month_df['month']==11]
JNorth_Adult_Site_2015_11_df.tail()

,sighting_date,species,latitude,longitude,number,month
30752,2015-11-27,Monarch Adult Sighted,32.237,-80.887,1,11
30753,2015-11-27,Monarch Adult Sighted,40.205,-75.530,1,11
30754,2015-11-28,Monarch Adult Sighted,28.179,-80.600,2,11
30755,2015-11-29,Monarch Adult Sighted,33.056,-96.733,2,11
30756,2015-11-29,Monarch Adult Sighted,33.889,-118.396,1,11


In [404]:
JNorth_Adult_Site_2015_11_df.shape

(423, 6)

In [405]:
JNorth_Adult_Site_2015_11_df = pd.DataFrame(JNorth_Adult_Site_2015_11_df)
JNorth_Adult_Site_2015_11_df.drop(['month'], axis=1, inplace=True)

In [406]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2015_11_df['latitude'] = JNorth_Adult_Site_2015_11_df['latitude'].astype(float)
JNorth_Adult_Site_2015_11_df['longitude'] = JNorth_Adult_Site_2015_11_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [407]:
geojson = df_to_geojson(JNorth_Adult_Site_2015_11_df, cols)

In [408]:
file = open("JNorth_Adult_Site_2015_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# December

In [409]:
JNorth_Adult_Site_2015_month_df =pd.DataFrame(JNorth_Adult_Site_2015_df)
JNorth_Adult_Site_2015_12_df = JNorth_Adult_Site_2015_month_df[JNorth_Adult_Site_2015_month_df['month']==12]
JNorth_Adult_Site_2015_12_df.tail()

,sighting_date,species,latitude,longitude,number,month
30819,2015-12-26,Monarch Adult Sighted,19.590,-100.266,1,12
30820,2015-12-26,Monarch Adult Sighted,30.047,-95.214,1,12
30821,2015-12-27,Monarch Adult Sighted,29.272,-95.729,1,12
30822,2015-12-27,Monarch Adult Sighted,32.237,-80.887,1,12
30823,2015-12-27,Monarch Adult Sighted,32.237,-80.860,1,12


In [410]:
JNorth_Adult_Site_2015_12_df.shape

(67, 6)

In [411]:
JNorth_Adult_Site_2015_12_df = pd.DataFrame(JNorth_Adult_Site_2015_12_df)
JNorth_Adult_Site_2015_12_df.drop(['month'], axis=1, inplace=True)

In [412]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Adult_Site_2015_12_df['latitude'] = JNorth_Adult_Site_2015_12_df['latitude'].astype(float)
JNorth_Adult_Site_2015_12_df['longitude'] = JNorth_Adult_Site_2015_12_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [413]:
geojson = df_to_geojson(JNorth_Adult_Site_2015_12_df, cols)

In [414]:
file = open("JNorth_Adult_Site_2015_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()